
![e-ORP](doc/img/e-orp-light-logo-240.png "e-ORP Logo")

Optimal Retirement Planner

Please see [the Github project page](https://github.com/dcurrie/e-ORP) for more information and to report problems.

## DISCLAIMER

This tool is freely offered for your enjoyment, but please be aware that:

1. It is new and undoubtedly has defects
2. I am not a financial planner 
3. I am not an expert in linear programming optimization

**Use at your own risk!**  

If you find what you belive is a defect in the code, please report it!

## Basic Usage Instructions

Click the ▶ run button in the menu ribbon above, fill in the blanks, play with the buttons!

You may be able to explore the output data from the last projection at: [data](data/explore.csv) If you 
turn off Simple mode using the control at the very bottom left of this window, the Juputer UI will show
tabs where both this notebook and the csv file can viewed.

For more detailed documentation, see [the User Guide](https://github.com/dcurrie/e-ORP/blob/main/doc/UserGuide.md)


In [ ]:
"""e-ORP"""

# Copyright (c) 2020-5 Doug Currie

import ipywidgets as widgets
import pandas as pd
import plotly.express as px
from IPython.display import display, Markdown, HTML
import pyscipopt
import math
import statistics
import traceback
# import numpy as np

from io import StringIO

import time # for awaiting SCIP output; is there a way to flush instead?

pd.options.display.max_columns = None # don't limit number of displayed columns
pd.options.display.precision = 3      # display up to 3 decimal places in dataframes

###############   Historical Data    ##############

hd = pd.read_csv('historical/rates.csv', index_col='year')  # 1970.. Rates of Return by year
min_hd_year = min(hd.index)
max_hd_year = max(hd.index)
# extend the dataframe so it the data "wraps around"
hd = pd.concat([hd,
                hd.copy().set_index(pd.Index((x+max_hd_year-min_hd_year+1 \
                                              for x in range(min_hd_year, max_hd_year+1)),
                                             name='year'))])

# ################# Output Widgets #################

# Use this for displaying error outputs
err_out = widgets.Output(layout={'border': '1px solid black'})

# Use this for displaying stdout outputs
drb_out = widgets.Output(layout={'border': '1px solid black'})

def display_warning(str):
    with err_out:
        display(HTML(f'<strong style="color:royalblue;"> WARNING! </strong> {str}'))

# ############## Input Widgets ##############

#	**Set Model Parameters**

#beright = widgets.Layout(display='flex', justify_content='flex-end')

layo4 = widgets.Layout(flex='1 1 auto', width='auto', margin='2px 8px 2px 2px') # [top/right/bottom/left] margins
layo2 = layo4

byear_box = widgets.BoundedIntText(
    value=2024,
    min=2000,
    max=9999,
    step=1,
    description='Base Year:',
    style={'description_width': '33%'},
    layout=layo4,
    disabled=False
)
def prc_widget(dflt, desc):
    return widgets.BoundedFloatText(
                value=dflt,
                min=0.0,
                max=99.0,
                step=0.1,
                description=desc,
                style={'description_width': '33%'},
                layout=layo4,
                disabled=False)
    
rorb_box = prc_widget(3.0, 'ROR Bonds (%)')
rors_box = prc_widget(7.0, 'ROR Stocks (%)')
dvdd_box = prc_widget(3.26,'Dividends (%)')
infl_box = prc_widget(2.0, 'Inflation (%)')
infs_box = prc_widget(4.0, 'SpendRate (%)')

hist_box = widgets.Dropdown(
    value='Use Values Below',
    options=['Use Values Below', *[str(y) for y in range(min_hd_year, max_hd_year+1)]],
    description='Historical Rates:',
    style={'description_width': '33%'},
    layout=layo4,
    disabled=False
)

# Essential Spending

layo9 = widgets.Layout(flex='4 1 auto', width='auto', margin='2px 8px 2px 2px') # [top/right/bottom/left] margins

def yrXX_widget():
    return widgets.BoundedIntText(
                value=2025,
                min=2025,
                max=9999,
                step=1,
                description='',
                style={'description_width': '0%'},
                layout=layo4,
                disabled=False)

yr01_box = yrXX_widget()
yr02_box = yrXX_widget()
yr03_box = yrXX_widget()
yr04_box = yrXX_widget()
yr05_box = yrXX_widget()
yr06_box = yrXX_widget()
yr07_box = yrXX_widget()
yr08_box = yrXX_widget()

def prXX_widget():
    return widgets.Dropdown(
                options=[('None', 0),
                         ('Essential Spending...:', 1),
                         ('One Time Expense:', 2),
                         ('One Time Income:', 3),
                        ],
                value=0,
                disabled=False,
                description='',
                layout=layo9,
                style={'description_width': '0%'},)

pr01_box = prXX_widget()
pr02_box = prXX_widget()
pr03_box = prXX_widget()
pr04_box = prXX_widget()
pr05_box = prXX_widget()
pr06_box = prXX_widget()
pr07_box = prXX_widget()
pr08_box = prXX_widget()

def vaXX_widget():
    return widgets.BoundedFloatText(
                value=0.0,
                min=0.0,
                max=999.9,
                step=1.0,
                description='',
                style={'description_width': '0%'},
                layout=layo4,
                disabled=False)

va01_box = vaXX_widget()
va02_box = vaXX_widget()
va03_box = vaXX_widget()
va04_box = vaXX_widget()
va05_box = vaXX_widget()
va06_box = vaXX_widget()
va07_box = vaXX_widget()
va08_box = vaXX_widget()

def txXX_box():
    return widgets.Text(
                value='',
                placeholder='',
                description='',
                style={'description_width': '0%'},
                #tooltip='Description (optional):',
                layout=layo9,
                disabled=False)

tx01_box = txXX_box()
tx02_box = txXX_box()
tx03_box = txXX_box()
tx04_box = txXX_box()
tx05_box = txXX_box()
tx06_box = txXX_box()
tx07_box = txXX_box()
tx08_box = txXX_box()

# Glide Path

def fra_widget(deflt,dsbl): 
    return widgets.BoundedFloatText(
                value=deflt,
                min=0.0,
                max=100.0,
                step=1.0,
                description='',
                style={'description_width': '0%'},
                layout=widgets.Layout(flex='1 1 auto', width='auto'),
                disabled=dsbl)

# base year
frasd_box = fra_widget(60.0, False)
frasr_box = fra_widget(60.0, False)
frasa_box = fra_widget(60.0, False)
frabd_box = fra_widget(40.0, False)
frabr_box = fra_widget(40.0, False)
fraba_box = fra_widget(40.0, False)
# horizon
frhsd_box = fra_widget(60.0, True)
frhsr_box = fra_widget(60.0, True)
frhsa_box = fra_widget(60.0, True)
frhbd_box = fra_widget(40.0, True)
frhbr_box = fra_widget(40.0, True)
frhba_box = fra_widget(40.0, True)

# widgets.Checkbox didn't work for glide path because setting the value in load_params didn't do anything

glide_box = widgets.Dropdown(
    options=[('Disabled', 0), ('Enabled', 1)],
    value=0,
    disabled=False,
    description='Glide Path:',
    layout=layo4,
    style={'description_width': '33%'},
)
def on_glide_change(change):
    if change['new']: # new glide_box.value:
        frhsd_box.disabled=False
        frhsr_box.disabled=False
        frhsa_box.disabled=False
        frhbd_box.disabled=False
        frhbr_box.disabled=False
        frhba_box.disabled=False
    else:
        # too fussy, and makes "what-if: use of the checkbox cumbersome
        # instead, ignore the horizon values if glide_box.value is False
        # frhsd_box.value=frasd_box.value
        # frhsr_box.value=frasr_box.value
        # frhsa_box.value=frasa_box.value
        # frhbd_box.value=frabd_box.value
        # frhbr_box.value=frabr_box.value
        # frhba_box.value=fraba_box.value
        frhsd_box.disabled=True
        frhsr_box.disabled=True
        frhsa_box.disabled=True
        frhbd_box.disabled=True
        frhbr_box.disabled=True
        frhba_box.disabled=True
        
glide_box.observe(on_glide_change, names='value')

def usd_widget(dflt,upb,desc):
    return widgets.BoundedFloatText(
                value=dflt,
                min=0.0,
                max=upb,
                step=1.0,
                description=desc,
                style={'description_width': '33%'},
                layout=layo4,
                disabled=False)
    
incn_box = usd_widget(50, 999.9, 'Spending $')
chty_box = usd_widget( 1, 999.9, 'Charity $')
xinc_box = usd_widget( 1, 999.9, 'Misc. Income $')
magib_box= usd_widget(42, 999.9, 'MAGI base year')
magip_box= usd_widget(40, 999.9, 'MAGI prior year')
ftab_box = usd_widget( 0, 9999.9, 'Plan Surplus') # Final Total Account Balance

spndm_box = widgets.Dropdown(
    options=[('Traditional (TSM)', 0), ('Changing Consumption', 1)],
    value=0,
    disabled=False,
    description='Spend Model:',
    style={'description_width': '33%'},
    layout=layo4,
)
xinr_box = widgets.BoundedFloatText(
    value=0.00,
    min= -100.0,
    max=999.0,
    step=1.0,
    description='Misc. Inc. δ (%)',# Δ
    style={'description_width': '33%'},
    layout=layo4,
    disabled=False
)
fstat_box = widgets.Dropdown(
    options=[('Single', 0), ('Married Filing Jointly', 1), ('Head of Household', 2)],
    value=1,
    disabled=False,
    description='Filing Status:',
    layout=layo4,
    style={'description_width': '33%'},
)

ssabr_box = widgets.Dropdown(
    description='SSA benefits are',
    options=[('reduced 23% in 2034', 1), ('maintained at current levels', 0)],
    value=True,
    style={'description_width': '33%'}, # 'initial'
    disabled=False,
    layout=layo4,
    indent=False
)
rothl_box = widgets.Dropdown(
    description='Roth conversions',
     # NOTE: the values correspond to the SCIP var we want to be zero
    options=[('None',                   'tax0'),
             ('Limit: 0% tax bracket',  'tax1'),
             ('Limit: 10% tax bracket', 'tax2'),
             ('Limit: 12% tax bracket', 'tax3'),
             ('Limit: 22% tax bracket', 'tax4'),
             ('Limit: 24% tax bracket', 'tax5'),
             ('Limit: 32% tax bracket', 'tax6'),
             ('Limit: 35% tax bracket', 'tax7'),
             ('Unlimited', 'unlimited'),
             ],
    value='unlimited',
    style={'description_width': '33%'}, # 'initial'
    disabled=False,
    layout=layo4,
    indent=False
)

# paired values for spouses
#
def age_widget(dflt, lb, ub, desc):
    return widgets.BoundedIntText(
                value=dflt,
                min=lb,
                max=ub,
                step=1,
                description=desc,
                style={'description_width': '40%'},
                layout=layo2,
                disabled=False)

aage1_box = age_widget(65, 0, 149, 'Age:')
aage2_box = age_widget(65, 0, 149, 'Age:')
fage1_box = age_widget(92, 0, 149, 'Age @ Horizon:')
fage2_box = age_widget(92, 0, 149, 'Age @ Horizon:')

def act_widget(dflt, desc):
    return widgets.BoundedFloatText(
    value=dflt,
    min=0,
    max=9999.9,
    step=1.0,
    description=desc,
    style={'description_width': '40%'},
    layout=layo2,
    disabled=False
)

atax1_box = act_widget( 50.0, 'After Tax $:')
atax2_box = act_widget( 50.0, 'After Tax $:')
bsis1_box = act_widget( 10.0, 'Cost Basis $:')
bsis2_box = act_widget( 10.0, 'Cost Basis $:')
taxd1_box = act_widget(100.0, 'Tax Deferred $:')
taxd2_box = act_widget(100.0, 'Tax Deferred $:')
roth1_box = act_widget(100.0, 'Roth IRA $:')
roth2_box = act_widget(100.0, 'Roth IRA $:')
ssar1_box = act_widget( 36.0, 'SSA/year $:')
ssar2_box = act_widget( 36.0, 'SSA/year $:')

refa1_box = age_widget(65, 62, 149, 'Ref. Age:')
refa2_box = age_widget(65, 62, 149, 'Ref. Age:')
reta1_box = age_widget(70, 62, 70, 'Claim Age:')
reta2_box = age_widget(65, 62, 70, 'Claim Age:')

def pen_widget():
    return  widgets.Dropdown(
                options=[('Fixed Payments', 0), ('COLA Payments', 1), ('Lump Sum Distribution', 2)],
                value=0,
                layout=layo2,
                disabled=False,
                description='Pension:',
                style={'description_width': '40%'},)

popt1_box = pen_widget()
popt2_box = pen_widget()

pens1_box = act_widget(  0.0, 'Pension $:')
pens2_box = act_widget(  0.0, 'Pension $:')

page1_box = age_widget(65, 62, 149, 'Claim Age:')
page2_box = age_widget(65, 62, 149, 'Claim Age:')

def ppt_widget():
    return widgets.BoundedFloatText(
                value=0.0,
                min=0.0,
                max=100.00,
                step=1.0,
                description='Survivor (%)',
                style={'description_width': '40%'},
                layout=layo2,
                disabled=False)
    
pinh1_box = ppt_widget()
pinh2_box = ppt_widget()

# csv file load/save name and buttons
#
pfname = widgets.Text(
    value='params/fname.csv',
    placeholder='filename.csv',
    description='File Name:',
    disabled=False
)
save_button = widgets.Button(
    description='Save Parameters Button',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Save Parameters',
    icon='download' # (FontAwesome names without the `fa-` prefix)
)
load_button = widgets.Button(
    description='Load Parameters Button',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Load Parameters',
    icon='upload' # (FontAwesome names without the `fa-` prefix)
)
param_buf = widgets.Textarea(
    value='',
    placeholder='',
    rows=5,
    description='Param CSV:',
    disabled=False
)
copy_button = widgets.Button(
    description='Dump to CSV',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Copy Parameters to Textarea',
    icon='copy' # (FontAwesome names without the `fa-` prefix)
)
paste_button = widgets.Button(
    description='Restore from CSV',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Paste Parameters from Textarea',
    icon='paste' # (FontAwesome names without the `fa-` prefix)
)

#### Load and Save Params ####

#       name,  widget, default
params = [ 
   ['byear', byear_box,  2024 ],
   ['rorb' , rorb_box ,  3.00 ],
   ['rors' , rors_box ,  7.00 ],
   ['dvdd' , dvdd_box ,  3.26 ],
   ['frasa', frasa_box,  0.60 ],
   ['frasr', frasr_box,  0.60 ],
   ['frasd', frasd_box,  0.60 ],
   ['fraba', fraba_box,  0.40 ],
   ['frabr', frabr_box,  0.40 ],
   ['frabd', frabd_box,  0.40 ],
   ['frhsa', frhsa_box,  0.60 ],
   ['frhsr', frhsr_box,  0.60 ],
   ['frhsd', frhsd_box,  0.60 ],
   ['frhba', frhba_box,  0.40 ],
   ['frhbr', frhbr_box,  0.40 ],
   ['frhbd', frhbd_box,  0.40 ],
   ['infl' , infl_box ,  0.02 ],
   ['infs' , infs_box ,  0.04 ],
   ['incn' , incn_box ,  50.0 ],
   ['spndm', spndm_box,  1.00 ],
   ['chty' , chty_box ,  0.00 ],
   ['xinc' , xinc_box ,  2024 ],
   ['xinr' , xinr_box ,  2024 ],
   ['magib', magib_box,  42.0 ],
   ['magip', magip_box,  40.0 ],
   ['fstat', fstat_box,  1.00 ],
   ['ftab' , ftab_box ,  0.00 ],
   ['aage1', aage1_box,  65.0 ],
   ['aage2', aage2_box,  65.0 ],
   ['fage1', fage1_box,  92.0 ],
   ['fage2', fage2_box,  92.0 ],
   ['atax1', atax1_box,  50.0 ],
   ['atax2', atax2_box,  50.0 ],
   ['bsis1', bsis1_box,  10.0 ],
   ['bsis2', bsis2_box,  10.0 ],
   ['taxd1', taxd1_box, 100.0 ],
   ['taxd2', taxd2_box, 100.0 ],
   ['roth1', roth1_box, 100.0 ],
   ['roth2', roth2_box, 100.0 ],
   ['ssar1', ssar1_box,  36.0 ],
   ['ssar2', ssar2_box,  36.0 ],
   ['refa1', refa1_box,  65.0 ],
   ['refa2', refa2_box,  65.0 ],
   ['reta1', reta1_box,  70.0 ],
   ['reta2', reta2_box,  65.0 ],
   ['popt1', popt1_box,  0.00 ],
   ['popt2', popt2_box,  0.00 ],
   ['pens1', pens1_box,  0.00 ],
   ['pens2', pens2_box,  0.00 ],
   ['page1', page1_box,  65.0 ],
   ['page2', page2_box,  65.0 ],
   ['pinh1', pinh1_box,  0.00 ],
   ['pinh2', pinh2_box,  0.00 ],
   ['yr01' , yr01_box,   2025 ],
   ['yr02' , yr02_box,   2025 ],
   ['yr03' , yr03_box,   2025 ],
   ['yr04' , yr04_box,   2025 ],
   ['yr05' , yr05_box,   2025 ],
   ['yr06' , yr06_box,   2025 ],
   ['yr07' , yr07_box,   2025 ],
   ['yr08' , yr08_box,   2025 ],
   ['pr01' , pr01_box,   0.00 ],
   ['pr02' , pr02_box,   0.00 ],
   ['pr03' , pr03_box,   0.00 ],
   ['pr04' , pr04_box,   0.00 ],
   ['pr05' , pr05_box,   0.00 ],
   ['pr06' , pr06_box,   0.00 ],
   ['pr07' , pr07_box,   0.00 ],
   ['pr08' , pr08_box,   0.00 ],
   ['va01' , va01_box,   0.00 ],
   ['va02' , va02_box,   0.00 ],
   ['va03' , va03_box,   0.00 ],
   ['va04' , va04_box,   0.00 ],
   ['va05' , va05_box,   0.00 ],
   ['va06' , va06_box,   0.00 ],
   ['va07' , va07_box,   0.00 ],
   ['va08' , va08_box,   0.00 ],
   ['tx01' , tx01_box,     '' ],
   ['tx02' , tx02_box,     '' ],
   ['tx03' , tx03_box,     '' ],
   ['tx04' , tx04_box,     '' ],
   ['tx05' , tx05_box,     '' ],
   ['tx06' , tx06_box,     '' ],
   ['tx07' , tx07_box,     '' ],
   ['tx08' , tx08_box,     '' ],
   ['glide', glide_box,  0.00 ],
   ['ssabr', ssabr_box,  True ],
   ['rothl', rothl_box, 'unlimited'],
   ['hist',  hist_box, 'Use Values Below']
]


# Warning: ridiculous kludge ahead
# Somewhere along the line, setting the value of a Dropdown from an onClick callback
# stopped working. It works when set manually from another Jupyter cell. Ugh.
# So, this workaround seems to work until I find the real solution: I link a 
# surrogate, undisplayed, BoundedIntText to the Dropdown, and update it instead.

def wgt_surrogate(wgt):
    surrogate = widgets.BoundedIntText(value=wgt.value,min=min(wgt._options_values),max=max(wgt._options_values))
    widgets.link((surrogate, 'value'), (wgt, 'value'))
    return surrogate

def txt_surrogate(wgt):
    surrogate = widgets.Text(value=wgt.value)
    widgets.link((surrogate, 'value'), (wgt, 'value'))
    return surrogate

surrogates = {
    'spndm': wgt_surrogate(spndm_box),
    'fstat': wgt_surrogate(fstat_box),
    'popt1': wgt_surrogate(popt1_box),
    'popt2': wgt_surrogate(popt2_box),
    'glide': wgt_surrogate(glide_box),
    'pr01' : wgt_surrogate(pr01_box ),
    'pr02' : wgt_surrogate(pr02_box ),
    'pr03' : wgt_surrogate(pr03_box ),
    'pr04' : wgt_surrogate(pr04_box ),
    'pr05' : wgt_surrogate(pr05_box ),
    'pr06' : wgt_surrogate(pr06_box ),
    'pr07' : wgt_surrogate(pr07_box ),
    'pr08' : wgt_surrogate(pr08_box ),
    'ssabr': wgt_surrogate(ssabr_box),
    'rothl': txt_surrogate(rothl_box),
    'hist' : txt_surrogate(hist_box),
}

def load_params(use_clip=False):
    """Read params from csv file or clipbpoard and load into widgets"""
    if use_clip:
        ps = pd.read_csv(filepath_or_buffer=StringIO(param_buf.value), index_col=0, keep_default_na=False)
    else:
        ps = pd.read_csv(filepath_or_buffer=pfname.value, index_col=0, keep_default_na=False)
    def pluck(key, dflt):
        try:
            return ps.loc[key]['0']
        except KeyError as e:
            display_warning(f'defaulting missing key {key} to {dflt}')
            return dflt
    err_out.clear_output()
    for name, wdgt, dflt in params:
        try:
            w = surrogates.get(name, wdgt)
            w.value = pluck(name, dflt)
        except Exception as e:
            display_warning(f'Unknown exception {e} trying {name} = {pluck(name, dflt)}')
    return ps

def save_params(use_clip):
    """Save params to csv file or clipsboard from widgets"""
    ps = pd.Series(index=[r[0] for r in params], data=[r[1].value for r in params])
    if use_clip:
        param_buf.value = ps.to_csv(None)
    else:
        ps.to_csv(path_or_buf=pfname.value)
    return ps

######## the User Inputs widget layout ########

save_button.on_click(lambda _: save_params(False))
load_button.on_click(lambda _: load_params(False))
copy_button.on_click(lambda _: save_params(True))
paste_button.on_click(lambda _: load_params(True))

becenter = widgets.Layout(display='flex', justify_content='center')

ratsl = widgets.HBox([widgets.Label('Account: ',layout=layo4),
                      widgets.Label('TaxDef',layout=layo4), 
                      widgets.Label('Roth',layout=layo4), 
                      widgets.Label('AftTax',layout=layo4)],
                     layout=widgets.Layout(margin='2px 8px 2px 2px'))
ratss = widgets.HBox([widgets.Label('Stocks %', layout=layo4),
                      frasd_box, frasr_box, frasa_box],
                     layout=widgets.Layout(margin='2px 8px 2px 2px'))
ratsb = widgets.HBox([widgets.Label(' Bonds %', layout=layo4),
                      frabd_box, frabr_box, fraba_box],
                     layout=widgets.Layout(margin='2px 8px 2px 2px'))
raths = widgets.HBox([widgets.Label('Stocks %', layout=layo4),
                      frhsd_box, frhsr_box, frhsa_box],
                     layout=widgets.Layout(margin='2px 8px 2px 2px'))
rathb = widgets.HBox([widgets.Label(' Bonds %', layout=layo4),
                      frhbd_box, frhbr_box, frhba_box],
                     layout=widgets.Layout(margin='2px 8px 2px 2px'))

phrlb = widgets.HBox([widgets.Label('Year',layout=layo4),
                      widgets.Label('Event',layout=layo9), 
                      widgets.Label('Amount',layout=layo4), #  (in $000s)
                      widgets.Label('Description (optional)',layout=layo9)],
                     layout=widgets.Layout(margin='2px 8px 2px'))
phr01 = widgets.HBox([yr01_box, pr01_box, va01_box, tx01_box],
                     layout=widgets.Layout(margin='2px 8px 2px'))
phr02 = widgets.HBox([yr02_box, pr02_box, va02_box, tx02_box],
                     layout=widgets.Layout(margin='2px 8px 2px'))
phr03 = widgets.HBox([yr03_box, pr03_box, va03_box, tx03_box],
                     layout=widgets.Layout(margin='2px 8px 2px'))
phr04 = widgets.HBox([yr04_box, pr04_box, va04_box, tx04_box],
                     layout=widgets.Layout(margin='2px 8px 2px'))
phr05 = widgets.HBox([yr05_box, pr05_box, va05_box, tx05_box],
                     layout=widgets.Layout(margin='2px 8px 2px'))
phr06 = widgets.HBox([yr06_box, pr06_box, va06_box, tx06_box],
                     layout=widgets.Layout(margin='2px 8px 2px'))
phr07 = widgets.HBox([yr07_box, pr07_box, va07_box, tx07_box],
                     layout=widgets.Layout(margin='2px 8px 2px'))
phr08 = widgets.HBox([yr08_box, pr08_box, va08_box, tx08_box],
                     layout=widgets.Layout(margin='2px 8px 2px'))

layo3c = widgets.Layout(grid_template_columns='30% 30% 40%',
                        border_bottom='1px solid black',
                        padding='2px 2px 8px 4px')

layo2c = widgets.Layout(grid_template_columns='35% 65%',
                        border_bottom='1px solid black',
                        padding='2px 2px 8px 4px')

layo1c = widgets.Layout(grid_template_columns='100%',
                        border_bottom='1px solid black',
                        padding='2px 2px 8px 4px')

winputs = widgets.VBox([
    widgets.GridBox([widgets.Label('e-ORP', style={
                        'font_weight':'bold',
                        'font_size':'large',
                        'text_color':'forestgreen',
                        }), widgets.Label(''),
                    widgets.Label('Set Model Parameters', style={
                        'font_weight':'bold',
                        'font_size':'large',
                        }), widgets.Label(''),
                 widgets.Label('Note: All dollar values in 000s...', style={'font_style':'italic',}),
                 widgets.Label(''),
                 byear_box, widgets.Label('account values below are at the end of the Base Year'),
                 ftab_box, widgets.Label('The Plan Surplus, or Final Total Account Balance'),
                ],
                layout=widgets.Layout(grid_template_columns='35% 65%')),

    widgets.GridBox([
                 widgets.Label('Person', layout=becenter),
                 widgets.Label('Spouse', layout=becenter),
                 widgets.Label(''),
                 aage1_box, aage2_box, widgets.Label('Age at end of base year'),
                 fage1_box, fage2_box, widgets.Label('Age at the Planning Horizon'),
                ],
                layout=layo3c), 

    widgets.GridBox([
                 widgets.Label('Asset Values', style={
                        # 'font_weight':'bold',
                        'font_size':'medium',
                        }), widgets.Label(''), widgets.Label(''),
                 widgets.Label('Person', layout=becenter),
                 widgets.Label('Spouse', layout=becenter),
                 widgets.Label(''),
                 atax1_box, atax2_box, widgets.Label('Non-retirement accounts'),
                 bsis1_box, bsis2_box, widgets.Label('Cost basis of above (for cap gains calc)'),
                 taxd1_box, taxd2_box, widgets.Label('Tax Deferred IRAs and 401(k)s'),
                 roth1_box, roth2_box, widgets.Label('Roth IRAs and 401(k)s'),
                ],
                layout=layo3c), 

    widgets.GridBox([
                 widgets.Label('Social Security', style={
                        # 'font_weight':'bold',
                        'font_size':'medium',
                        }), widgets.Label(''), widgets.Label(''),
                 widgets.Label('Person', layout=becenter),
                 widgets.Label('Spouse', layout=becenter),
                 widgets.Label(''),
                 ssar1_box, ssar2_box, widgets.Label('Social Security income, annual'),
                 refa1_box, refa2_box, widgets.Label('Reference age of Social Security income'),
                 reta1_box, reta2_box, widgets.Label('Age when Social Security is claimed'),
                ],
                style={'padding-bottom': '40px'},
                layout=layo3c), 

    widgets.GridBox([
                 widgets.Label('Pensions', style={
                        # 'font_weight':'bold',
                        'font_size':'medium',
                        }), widgets.Label(''), widgets.Label(''),
                 widgets.Label('Person', layout=becenter),
                 widgets.Label('Spouse', layout=becenter),
                 widgets.Label(''),
                 popt1_box, popt2_box, widgets.Label('Taxable Pension or Annuity Type'),
                 pens1_box, pens2_box, widgets.Label('Annual payment for fixed or COLA; total for lump'),
                 page1_box, page2_box, widgets.Label('Age when pension or annuity is claimed'),
                 pinh1_box, pinh2_box, widgets.Label('Percentage of benefit for surviving spouse'),
                ],
                layout=layo3c), 

    widgets.GridBox([
                 widgets.Label('Other Income', style={
                        # 'font_weight':'bold',
                        'font_size':'medium',
                        }), widgets.Label(''),
                 xinc_box, widgets.Label('miscellaneous annual income (Inherited IRA, etc.)'),
                 xinr_box, widgets.Label('rate of change (+/-) applied to the miscellaneous income'),
                ],
                layout=layo2c),
    
    widgets.GridBox([
                 widgets.Label('Phases of Retirement: Spending Model', style={
                        # 'font_weight':'bold',
                        'font_size':'medium',
                        }), widgets.Label(''),
                 infs_box, widgets.Label('annual inflation rate applied to spending, percentage'),
                 incn_box, widgets.Label('annual disposable income expected for spending, after taxes'),
                 spndm_box, widgets.Label('spending model; above two inputs are parameters to the model'),
                ],
                layout=layo2c),
    
    widgets.GridBox([
                 widgets.Label('Phases of Retirement: Essential Spending and Extraordinary Expenses', style={
                        # 'font_weight':'bold',
                        'font_size':'medium',
                        }),
                 phrlb, phr01, phr02, phr03, phr04, phr05, phr06, phr07, phr08,
                ],
                layout=layo1c),
    
    widgets.GridBox([
                 widgets.Label('Asset Allocation & Glide Path', style={
                        # 'font_weight':'bold',
                        'font_size':'medium',
                        }), widgets.Label(''),
                 ratsl, widgets.Label('Note: different ratios between account types may lead to unexpected behavior!'),
                 ratss, widgets.Label('percentage of each account in stock investments, base year'),
                 ratsb, widgets.Label('percentage of each account in bond investments, base year'),
                 glide_box, widgets.Label(''),
                 raths, widgets.Label('percentage of each account in stock investments, horizon year'),
                 rathb, widgets.Label('percentage of each account in bond investments, horizon year'),
                ],
                layout=layo2c),
    
    widgets.GridBox([
                 widgets.Label('Rates', style={
                        # 'font_weight':'bold',
                        'font_size':'medium',
                        }), widgets.Label(''),
                 infl_box, widgets.Label('annual inflation rate for SSA payments & tax brackets, percentage'),
                 hist_box, widgets.Label('rates from years beginning at historical year replace the following rates'),
                 rorb_box, widgets.Label('rate of return on bond portion of accounts, annual percentage'),
                 rors_box, widgets.Label('rate of return on stock portion of accounts, annual percentage'),
                 dvdd_box, widgets.Label('annual dividend rate on stock portion of non-retirement account'),
                ],
                layout=layo2c),

    widgets.GridBox([
                 widgets.Label('Taxes', style={
                        # 'font_weight':'bold',
                        'font_size':'medium',
                        }), widgets.Label(''),
                 magib_box, widgets.Label('Modified Adjusted Gross Income (MAGI) in base year'),
                 magip_box, widgets.Label('MAGI in the year prior to the base year, needed for IRMAA'),
                 chty_box,  widgets.Label('annual charitable contributions, used for QCD calculation'),
                 fstat_box, widgets.Label('Federal Income Tax filing status'),
                ],
                layout=layo2c),
    
    widgets.GridBox([
                 widgets.Label('Modeling Assumptions and Constraints', style={
                        # 'font_weight':'bold',
                        'font_size':'medium',
                        }), widgets.Label(''),
                 ssabr_box, widgets.Label('Do you have confidence in the government to fix this?'),
                 rothl_box, widgets.Label('Roth Conversions limit, perhaps to avoid ACA cliff or IRMAA'),
                ],
                layout=layo2c),
    
    widgets.VBox([widgets.Label('Parameter Save & Load', style={'font_size':'medium',}),
                  widgets.HBox([pfname, save_button, load_button],
                               layout=widgets.Layout(grid_template_columns='35% 65%')),
                  widgets.HBox([param_buf, copy_button, paste_button],
                               layout=widgets.Layout(grid_template_columns='35% 65%'))],
                layout=layo1c),
], layout=widgets.Layout(border='1px solid black'))


# ##############      Tax Data       ##############

# Income tax rates

tax_rates =    [ 0.100,   0.120,   0.220,   0.240,   0.320,   0.350,   0.370 ]

# Data for 2025; update by applying inflation rate for projections
# each entry in the ordered dict is top of income bracket in 000s

tax_brackets = [[11.925, 48.475, 103.350, 197.300, 250.525, 626.350], # Single
                [23.850, 96.950, 206.700, 394.600, 501.050, 751.600], # MFJ Married Filing Jointly
                [17.000, 64.850, 103.350, 197.300, 250.500, 626.350]] # Head of Household

# Capital Gains tax rates

cgt_rates =     [0.000, 0.150, 0.200]

cgt_brackets =  [[48.35, 600.05], # Single
                 [96.70, 533.40], # MFJ Married Filing Jointly
                 [64.75, 566.70]] # Head of Household

std_deductions = [15.750, 31.500, 23.125] # Single, MFJ, HoH

# additional standard deduction based on age >= 65 or blindness is $1,600 
#
# OBBBA adds an additional $6,000 per person deduction based on age >= 65 through 2028
# taxpayers with modified adjusted gross income over $75,000 ($150,000 for joint filers).
#
addl_obbba_deduction_age65 = 6.000
addl_deduction_age65 = 1.600

base_year_irs_brackets = 2025

# filing_status: 0: Single, 1: MFJ, 2: Head of Household
#
def tax_bucket_n_size(year, n, age1, age2, filing_status=1, rate_infla=0.02):
    infla = 1.0
    if year < base_year_irs_brackets:
        with err_out:
            print('FAIL: no tax info for years before 2025')
        year = base_year_irs_brackets
    else:
        infla = (1.0 + rate_infla) ** (year - base_year_irs_brackets)
    size = 0.0 # the size of this tax bucket
    if n == 0:
        size = std_deductions[filing_status] * infla
        if age1 >= 65:
            size += addl_deduction_age65 * infla
            #if year <= 2028: size += addl_obbba_deduction_age65
        if filing_status == 1 and age2 >= 65:
            size += addl_deduction_age65 * infla
            #if year <= 2028: size += addl_obbba_deduction_age65
    elif n == 1:
        size = tax_brackets[filing_status][0] * infla
    else:
        size = (tax_brackets[filing_status][n-1] - tax_brackets[filing_status][n-2]) * infla
    return size

def cgt_bucket_n_size(year, n, filing_status=1, rate_infla=0.02):
    infla = 1.0
    if year < base_year_irs_brackets:
        with err_out:
            print('FAIL: no tax info for years before 2025')
        year = base_year_irs_brackets
    else:
        infla = (1.0 + rate_infla) ** (year - base_year_irs_brackets)
    size = 0.0 # the size of this tax bucket
    if n == 0:
        size = cgt_brackets[filing_status][0] * infla
    else:
        size = (tax_brackets[filing_status][1] - tax_brackets[filing_status][0]) * infla
    return size

def obbba_pax_in_year(year, age1, age2):
    return 0 if year > 2028 else ((1 if age1 >= 65 else 0) + (1 if age2 >= 65 else 0))

# IRMAA 2025 rates and brackets

IRMAA_buks = [[106, (133 - 106), (167 - 133), (200 - 167), (500 - 200), 9999], # Single
              [212, (266 - 212), (334 - 266), (400 - 334), (750 - 400), 9999], # MFJ Married Filing Jointly
              [106, (133 - 106), (167 - 133), (200 - 167), (500 - 200), 9999]] # Head of Household

IRMAA_chgs =  [(12 *  185.0        ) / 1000,
               (12 * (259.0 + 13.7)) / 1000,
               (12 * (370.0 + 35.3)) / 1000,
               (12 * (480.9 + 57.0)) / 1000,
               (12 * (591.9 + 78.6)) / 1000,
               (12 * (628.9 + 85.8)) / 1000]

def IRMAA_buk_n_size(year, n, filing_status=1, rate_infla=0.02):
    infla = (1.0 + rate_infla) ** (year - base_year_irs_brackets)
    return IRMAA_buks[filing_status][n] * infla

def IRMAA_chg_n_size(year, n, pax, rate_infla=0.02):
    infla = (1.0 + rate_infla) ** (year - base_year_irs_brackets)
    return pax * (IRMAA_chgs[n] - (0 if n == 0 else IRMAA_chgs[n-1])) * infla

# RMD table

RMD_divisor = [ 27.4, 26.5, 25.5, 24.6, 23.7, 22.9, 22.0, 21.1, 20.2, 19.4, 18.5,
                17.7, 16.8, 16.0, 15.2, 14.4, 13.7, 12.9, 12.2, 11.5, 10.8, 10.1,
                 9.5,  8.9,  8.4,  7.8,  7.3,  6.8,  6.4,  6.0,  5.6,  5.2,  4.9,
                 4.6,  4.3,  4.1,  3.9,  3.7,  3.5,  3.4,  3.3,  3.1,  3.0,  2.9,
                 2.8,  2.7,  2.5,  2.3,  2.0]

# QCD rules

annual_QCD_limit_pp = 108.0 # in base_year_irs_brackets, per person over 70.5

# ############## The Planning Data Dictionary ##############

# The data dictionary (`dd` in the code) is used for inputs to the model, and outputs from the
# model. It is stored at the completion of each projection as a csv file. 
# 
# The `dd` is indexed by plan year, with year 0 being the "base year." Each row of the `dd` 
# represents one plan year, and each column a parameter.
#
# The base year has only inputs to the model. As such, there are unused cells in row 0 of 
# the output columns. These cells are used to hold miscellaneous inputs to the model, such as 
# rates of return and inflation rate. 
# 
# The `squirrel_map` is used to map names of these miscellaneous parameters and column names.
# The `set_nut` and `get_nut` functions are used to access the parameters.

squirrel_map = {'inflation':     'IRMAA-buk0',
                'filing_status': 'tax_bracket',
                'MAGI_prebase':  'QCD_limit',
                'orp_mode':      'IRMAA-buk1',
                'orp_objtv':     'IRMAA-buk2',
                'eoplan_spouse': 'IRMAA-buk3',
                'min_realized':  'IRMAA-buk4',
                'gap_limit':     'IRMAA-buk5',
                'scip_status':   'IRMAA-chg0',
                'scip_stage':    'IRMAA-chg1',
                'scip_gap':      'IRMAA-chg2',
                'scip_time':     'IRMAA-chg3',
                'time_limit':    'IRMAA-chg4',
                'Roth_conv_max': 'IRMAA-chg5',
                'e-ORP_version': 'from_aTax'}

def set_nut(dd, parm, val):
    dd[squirrel_map[parm]][0] = val

def get_nut(dd, parm):
    return dd[squirrel_map[parm]][0]

def make_planning_datadict(test_mode, historical_year_for_rates=None):
    """Create the datadict to be used by OORPy, populated from the widgets"""
    
    age1 = aage1_box.value
    age2 = aage2_box.value
    taxd2 = taxd2_box.value
    roth2 = roth2_box.value

    fstat = fstat_box.value

    # Sanity and consistency checks
    
    if fstat != 1 and (roth2 != 0 or taxd2 != 0 or age2 != 0):
        display_warning(f'Filing status is not MFJ but spouse values are not zero')
        display_warning(f'Defaulting Spouse Roth and TaxD from {roth2} {taxd2} to zero.')
        age2 = 0
        taxd2 = 0
        roth2 = 0

    # Base year asset allocation 
    fraba = fraba_box.value
    frasa = frasa_box.value
    if (fraba + frasa) > 100.0:
        display_warning(f'Percent of savings in stocks {frasa: 2.3f}% + bonds {fraba: 2.3f}% exceeds 100%')
        redu = 100.0 / (fraba + frasa)
        fraba = fraba * redu
        frasa = frasa * redu
        display_warning(f'Reducing percentages proportionally to {frasa: 2.3f}% stocks + {fraba: 2.3f}% bonds')
    frabr = frabr_box.value
    frasr = frasr_box.value
    if (frabr + frasr) > 100.0:
        display_warning(f'Percent of savings in stocks {frasr: 2.3f}% + bonds {frabr: 2.3f}% exceeds 100%')
        redu = 100.0 / (frabr + frasr)
        frabr = frabr * redu
        frasr = frasr * redu
        display_warning(f'Reducing percentages proportionally to {frasr: 2.3f}% stocks + {frabr: 2.3f}% bonds')
    frabd = frabd_box.value
    frasd = frasd_box.value
    if (frabd + frasd) > 100.0:
        display_warning(f'Percent of savings in stocks {frasd: 2.3f}% + bonds {frabd: 2.3f}% exceeds 100%')
        redu = 100.0 / (frabd + frasd)
        frabd = frabd * redu
        frasd = frasd * redu
        display_warning(f'Reducing percentages proportionally to {frasd: 2.3f}% stocks + {frabd: 2.3f}% bonds')
    # Horizon year asset allocation 
    frhba = frhba_box.value
    frhsa = frhsa_box.value
    if (frhba + frhsa) > 100.0:
        display_warning(f'Percent of savings in stocks {frhsa: 2.3f}% + bonds {frhba: 2.3f}% exceeds 100%')
        redu = 100.0 / (frhba + frhsa)
        frhba = frhba * redu
        frhsa = frhsa * redu
        display_warning(f'Reducing percentages proportionally to {frhsa: 2.3f}% stocks + {frhba: 2.3f}% bonds')
    frhbr = frhbr_box.value
    frhsr = frhsr_box.value
    if (frhbr + frhsr) > 100.0:
        display_warning(f'Percent of savings in stocks {frhsr: 2.3f}% + bonds {frhbr: 2.3f}% exceeds 100%')
        redu = 100.0 / (frhbr + frhsr)
        frhbr = frhbr * redu
        frhsr = frhsr * redu
        display_warning(f'Reducing percentages proportionally to {frhsr: 2.3f}% stocks + {frhbr: 2.3f}% bonds')
    frhbd = frhbd_box.value
    frhsd = frhsd_box.value
    if (frhbd + frhsd) > 100.0:
        display_warning(f'Percent of savings in stocks {frhsd: 2.3f}% + bonds {frhbd: 2.3f}% exceeds 100%')
        redu = 100.0 / (frhbd + frhsd)
        frhbd = frhbd * redu
        frhsd = frhsd * redu
        display_warning(f'Reducing percentages proportionally to {frhsd: 2.3f}% stocks + {frhbd: 2.3f}% bonds')

    dvdd = dvdd_box.value
    rors = rors_box.value
    if dvdd > rors_box.value:
        display_warning(f'Dividends of {dvdd: 2.3f}% exceed stocks total return {rors: 2.3f}%')
        display_warning(f'Reducing the annual dividend rate to {rors/2: 2.3f}%')
        dvdd = rors/2

    fage1 = fage1_box.value
    fage2 = fage2_box.value
    if fage1 <= age1:
        display_warning(f'The person 1 planning horizon {fage1} is being extended to provide one planning year')
        fage1 = age1 + 1
    if age2 != 0 and fage2 <= age2:
        display_warning(f'The person 2 planning horizon {fage2} is being extended to provide one planning year')
        display_warning(f'You can use a base year age of 0 to exclude person 2 from the plan')
        fage2 = age2 + 1

    # Planning Horizons
    
    byear = byear_box.value
    years = 24
    y_spouse_leaves_plan = years + 1
    if age2 == 0:
        years = 1 + fage1 - age1
        y_spouse_leaves_plan = years + 1
    else:
        years1 = 1 + fage1 - age1
        years2 = 1 + fage2 - age2
        years = max(years1, years2)
        y_spouse_leaves_plan = min(years1, years2)

    idx = range(byear, byear + years)
    
    infl = infl_box.value / 100
    infs = infs_box.value / 100

    def ssa_calc(y):
        """Calculate SSA annual income based on age and initial data from the widgets"""
        e = age1 + y
        j = age2 + y
        reduce_SSAb = (ssabr_box.value == 1)
        e_ssa = ssar1_box.value * ((1.0 + infl) ** (e - refa1_box.value)) if e > reta1_box.value else 0.0
        j_ssa = ssar2_box.value * ((1.0 + infl) ** (j - refa2_box.value)) if j >= reta2_box.value else 0.0
        t_ssa = j_ssa + e_ssa
        if y >= y_spouse_leaves_plan:
            t_ssa = max(j_ssa, e_ssa)
        return t_ssa * (0.77 if (reduce_SSAb and ((byear + y) >= 2034)) else 1.0)

    def pension_calc(y):
        """Calculate pension annual income based on age and initial data from the widgets"""
        e = age1 + y
        j = age2 + y
        e_p = 0 if (popt1_box.value == 2 or e < page1_box.value) else \
                    pens1_box.value if popt1_box.value == 0 else \
                        pens1_box.value * ((1.0 + infl) ** (e - page1_box.value)) # use y for number of years of COLA?
        j_p = 0 if (popt2_box.value == 2 or j < page2_box.value) else \
                    pens2_box.value if popt2_box.value == 0 else \
                        pens2_box.value * ((1.0 + infl) ** (j - page2_box.value))
        # Inherited pension calculation
        e_p = e_p * (1.0 if e <= fage1 else (pinh1_box.value / 100))
        j_p = j_p * (1.0 if j <= fage2 else (pinh2_box.value / 100))
        return (j_p + e_p)

    dd = {'e':              range(age1, age1 + years),
          'j':              range(age2, age2 + years),
          'afterTax':       [0.0 for x in idx],
          #'aTax_basis':     [0.0 for x in idx],
          'e_Roth':         [0.0 for x in idx],
          'e_Taxd':         [0.0 for x in idx],
          'j_Roth':         [0.0 for x in idx],
          'j_Taxd':         [0.0 for x in idx],
          'e_RMD':          [0.0 for x in idx],
          'j_RMD':          [0.0 for x in idx],
          'to_aTax':        [0.0 for x in idx],
          'from_aTax':      [0.0 for x in idx],
          'from_eRoth':     [0.0 for x in idx],
          'from_jRoth':     [0.0 for x in idx],
          'from_eTaxd':     [0.0 for x in idx],
          'from_jTaxd':     [0.0 for x in idx],
          'e_RothConv' :    [0.0 for x in idx],
          'j_RothConv' :    [0.0 for x in idx],
          'SSA_income':     [ssa_calc(y) for y in range(len(idx))],
          'pension_income': [pension_calc(y) for y in range(len(idx))],
          'misc_income':    [max(0,round(xinc_box.value * (1.0 + xinr_box.value / 100) ** y, 3)) for y in range(len(idx))],
          'charity':        [chty_box.value * (1.0 + infl) ** y for y in range(len(idx))],
          'e_Taxd_in':      [0.0 for x in idx],
          'j_Taxd_in':      [0.0 for x in idx],
          'spend_essence':  [0.0 for x in idx],
          'taxfree_income': [0.0 for x in idx],
          'auto_income':    [0.0 for x in idx], # sum of previous five
          'disp_income':    [incn_box.value * (1.0 + infs) ** y for y in range(len(idx))],
          'taxable_income': [0.0 for x in idx],
          'IRMAA':          [0.0 for x in idx],
          'dividends':      [0.0 for x in idx],
          'capgains':       [0.0 for x in idx],
          'caplosss':       [0.0 for x in idx],
          'QCD_limit':      [0.0 for x in idx],
          'QCD':            [0.0 for x in idx],
          'income_tax':     [0.0 for x in idx],
          'tax_bracket':    [0.0 for x in idx],
          'cgains_rate':    [0.0 for x in idx],
          'MAGI':           [0.0 for x in idx],
          # 
          'ror_bonds':      [rorb_box.value / 100 for x in idx],
          'ror_stock':      [rors_box.value / 100 for x in idx],
          'dvd_stock':      [dvdd  / 100 for x in idx],
          'frac_bonds_a':   [fraba / 100 for x in idx],
          'frac_stock_a':   [frasa / 100 for x in idx],
          'frac_bonds_r':   [frabr / 100 for x in idx],
          'frac_stock_r':   [frasr / 100 for x in idx],
          'frac_bonds_d':   [frabd / 100 for x in idx],
          'frac_stock_d':   [frasd / 100 for x in idx],
          # asset allocations for afterTax account
          'aTax__cash':     [0.0 for x in idx],
          'aTax_bonds':     [0.0 for x in idx],
          'aTax_basis':     [0.0 for x in idx],
          'aTax_unrlz':     [0.0 for x in idx],
          'fm_aTax__cash':  [0.0 for x in idx],
          'fm_aTax_bonds':  [0.0 for x in idx],
          'fm_aTax_basis':  [0.0 for x in idx],
          'fm_aTax_unrlz':  [0.0 for x in idx],
          'to_aTax__cash':  [0.0 for x in idx],
          'to_aTax_bonds':  [0.0 for x in idx],
          'to_aTax_basis':  [0.0 for x in idx],
          'to_aTax_unrlz':  [0.0 for x in idx],
          # 
          'e_RMD_factor':  [(0.0 if e < 73 else 1 / RMD_divisor[e - 72])
                            for e in range(age1, age1 + years)],
          'j_RMD_factor':  [(0.0 if j < 73 else 1 / RMD_divisor[j - 72])
                            for j in range(age2, age2 + years)],
          # Tax buckets
          'tax0':          [0.0 for x in idx],
          'tax1':          [0.0 for x in idx],
          'tax2':          [0.0 for x in idx],
          'tax3':          [0.0 for x in idx],
          'tax4':          [0.0 for x in idx],
          'tax5':          [0.0 for x in idx],
          'tax6':          [0.0 for x in idx],
          'cgt0':          [0.0 for x in idx],
          'cgt15':         [0.0 for x in idx],
          'obbba_pax':     [0 for x in idx],
          # IRMAA buckets (brackets and surcharges per bracket)
          'IRMAA-buk0':    [0.0 for x in idx],
          'IRMAA-buk1':    [0.0 for x in idx],
          'IRMAA-buk2':    [0.0 for x in idx],
          'IRMAA-buk3':    [0.0 for x in idx],
          'IRMAA-buk4':    [0.0 for x in idx],
          'IRMAA-buk5':    [0.0 for x in idx],
          'IRMAA-chg0':    [0.0 for x in idx],
          'IRMAA-chg1':    [0.0 for x in idx],
          'IRMAA-chg2':    [0.0 for x in idx],
          'IRMAA-chg3':    [0.0 for x in idx],
          'IRMAA-chg4':    [0.0 for x in idx],
          'IRMAA-chg5':    [0.0 for x in idx],
          'IRMAA-bins':    [0   for x in idx],
          #
          'spend_δ':       [1.0 + infs for y in idx],   
          'surplus':       [ftab_box.value * (1.0 + infl) ** y for y in range(len(idx))],
          'net_pretax':    [0.0 for x in idx],
          'net_postax':    [0.0 for x in idx],
          'year':          idx,
        }

    # squirrel away miscellaneous inputs for reference by solver and to record with the dd'd csv dump
    set_nut(dd, 'inflation'    , infl)
    set_nut(dd, 'filing_status', fstat)
    set_nut(dd, 'eoplan_spouse', y_spouse_leaves_plan)
    set_nut(dd, 'MAGI_prebase' , magip_box.value)
    set_nut(dd, 'Roth_conv_max', rothl_box.value) # limit Roth Conversions
    set_nut(dd, 'e-ORP_version', 0.5)

    dd['e_Taxd'][0] = taxd1_box.value
    dd['j_Taxd'][0] = taxd2
    dd['e_Roth'][0] = roth1_box.value
    dd['j_Roth'][0] = roth2
    dd['MAGI'][0]   = magib_box.value
    # AAA
    dd['afterTax'][0]   = atax1_box.value + atax2_box.value
    dd['aTax_basis'][0] = bsis1_box.value + bsis2_box.value
    # loss # limit aTax_basis to stock portion of afterTax
    # loss if dd['aTax_basis'][0] > (dd['afterTax'][0] * frasa / 100):
    # loss     display_warning(f'AfterTax account basis exceeds total stock position {dd["aTax_basis"][0]} > {dd["afterTax"][0] * frasa / 100}')
    # loss     display_warning(f'Ignoring unrealized loss of {dd["aTax_basis"][0] - dd["afterTax"][0] * frasa / 100}')
    # loss dd['aTax_basis'][0] = min(dd['aTax_basis'][0], dd['afterTax'][0] * frasa / 100)
    # or report it as a warning
    if dd['aTax_basis'][0] > (dd['afterTax'][0] * frasa / 100):
        display_warning(f'AfterTax account basis exceeds total stock position {dd["aTax_basis"][0]} > {dd["afterTax"][0] * frasa / 100}')
        display_warning(f'This will result in a starting unrealized gain of {dd["afterTax"][0] * frasa / 100 - dd["aTax_basis"][0]}')
    dd['aTax_bonds'][0] = dd['afterTax'][0] * fraba / 100
    dd['aTax_unrlz'][0] = dd['afterTax'][0] * frasa / 100 - dd['aTax_basis'][0]
    dd['aTax__cash'][0] = dd['afterTax'][0] * (100.0 - frasa - fraba)

    # fix RMD_factors for unequal planning horizons
    if y_spouse_leaves_plan < years:
        for y in range(y_spouse_leaves_plan, years):
            # use remaining spouse's RMD_factor
            if dd['e'][y] > fage1:
                dd['e_RMD_factor'][y] = dd['j_RMD_factor'][y]
            else: # dd['j'][y] > fage2:
                dd['j_RMD_factor'][y] = dd['e_RMD_factor'][y]

    # lump sum pensions
    if popt1_box.value == 2:
        # The pension is added to 'e_Taxd' at the year when e reaches page1_box.value
        y = page1_box.value - age1
        if y > 0 and y < years:
            if page1_box.value > fage1:
                display_warning(f'Lump sum pension distribution of {pens1_box.value} not in plan horizon at year {y}')
                display_warning(f'Reducing to {pens1_box.value * (pinh1_box.value / 100)}, the spousal benefit')
                dd['e_Taxd_in'][y] = pens1_box.value * (pinh1_box.value / 100)
            else:
                dd['e_Taxd_in'][y] = pens1_box.value
        else:
            display_warning(f'Lump sum pension distribution of {pens1_box.value} not in plan horizon at year {y}')
    if popt2_box.value == 2:
        # The pension is added to 'j_Taxd' at the year when j reaches page2_box.value
        y = page2_box.value - age2
        if y > 0 and y < years:
            if page2_box.value > fage2:
                display_warning(f'Lump sum pension distribution of {pens2_box.value} not in plan horizon at year {y}')
                display_warning(f'Reducing to {pens2_box.value * (pinh2_box.value / 100)}, the spousal benefit')
                dd['j_Taxd_in'][y] = pens2_box.value * (pinh2_box.value / 100)
            else:
                dd['j_Taxd_in'][y] = pens2_box.value
        else:
            display_warning(f'Lump sum pension distribution of {pens2_box.value} not in plan horizon at year {y}')

    # Phases of Retirement
    def pormt_event(year, event, amount):
        if event == 0: # ('None', 0)
            return
        if year <= byear or year > byear + years:
            display_warning(f'Phase of Retirement Event in {year} is beyond planning horizon, and ignored')
            return
        else:
            evy = year - byear
        if event == 1: # ('Ongoing Essential Spending Set To:', 1)
            for y in range(evy,years):
                dd['spend_essence'][y] = amount * (1.0 + infl) ** y
        elif event == 2: # ('One Time Expense:', 2)
            dd['spend_essence'][evy] += amount * (1.0 + infl) ** evy
        elif event == 3: # ('One Time Income:', 3)
            dd['taxfree_income'][evy] += amount * (1.0 + infl) ** evy

    pormt_event(yr01_box.value, pr01_box.value, va01_box.value)
    pormt_event(yr02_box.value, pr02_box.value, va02_box.value)
    pormt_event(yr03_box.value, pr03_box.value, va03_box.value)
    pormt_event(yr04_box.value, pr04_box.value, va04_box.value)
    pormt_event(yr05_box.value, pr05_box.value, va05_box.value)
    pormt_event(yr06_box.value, pr06_box.value, va06_box.value)
    pormt_event(yr07_box.value, pr07_box.value, va07_box.value)
    pormt_event(yr08_box.value, pr08_box.value, va08_box.value)

    # compute spending deltas based on David Blanchett's "Estimating the True Cost of Retirement"
    # and incorporating the input spending inflation rate
    # We adjust Blanchett's formula for the 2012..2025 CPI-E delta
    # June 2012	246.716
    # June 2025	352.769
    # so use 0.0066 * ln(targetspend * 246.716 / 352.769) 
    # = 0.0066 * ln(targetspend) + 0.0066 * ln(246.716 / 352.769)
    # 0.546 + 0.0066 * math.log(246.716 / 352.769) = 0.54364
    
    if spndm_box.value == 1:
        # handle individual vs married
        age1s = dd['e']
        age2s = dd['j'] if age2 != 0 else dd['e']
        for y in range(1,years):
            dd['spend_δ'][y] = dd['spend_δ'][0] \
                                * (1.0 + ((0.00008 * (age1s[y] * age2s[y]))
                                           - (0.0125 * (age1s[y] + age2s[y]) / 2)
                                           - 0.0066 * math.log(dd['disp_income'][0] * 1000.0)
                                           + 0.54364)) # Blanchett's 2013 number was 0.546
            if y == y_spouse_leaves_plan:
                # reduce spending by 25% when spouse leaves plan
                dd['spend_δ'][y] = 0.75 * dd['spend_δ'][y]
            dd['disp_income'][y] = dd['spend_δ'][y] * dd['disp_income'][y-1]

    # compute tax brackets and number of people (pax) qualifying for OBBBA retirement $6000 kicker
    # compute IRMAA buckets and surcharges
    for y in range(1,years):
        fs = fstat if y < y_spouse_leaves_plan else 0 # Single after spouse leaves plan
        ag1 = dd['e'][y] if dd['e'][y] <= fage1 else 0 # use age 0 for spouse who leaves plan
        ag2 = dd['j'][y] if dd['j'][y] <= fage2 else 0 # since the only purpose is to check >= 65 or >= 70
        dd['tax0'][y] = tax_bucket_n_size(dd['year'][y], 0, ag1, ag2, fs, infl)
        dd['tax1'][y] = tax_bucket_n_size(dd['year'][y], 1, ag1, ag2, fs, infl)
        dd['tax2'][y] = tax_bucket_n_size(dd['year'][y], 2, ag1, ag2, fs, infl)
        dd['tax3'][y] = tax_bucket_n_size(dd['year'][y], 3, ag1, ag2, fs, infl)
        dd['tax4'][y] = tax_bucket_n_size(dd['year'][y], 4, ag1, ag2, fs, infl)
        dd['tax5'][y] = tax_bucket_n_size(dd['year'][y], 5, ag1, ag2, fs, infl)
        dd['tax6'][y] = tax_bucket_n_size(dd['year'][y], 6, ag1, ag2, fs, infl)
        dd['cgt0'][y] = dd['tax0'][y] + cgt_bucket_n_size(dd['year'][y], 0, fs, infl)
        dd['cgt15'][y] = cgt_bucket_n_size(dd['year'][y], 1, fs, infl)
        dd['obbba_pax'][y] = obbba_pax_in_year(dd['year'][y], ag1, ag2)
        IRMAA_pax = (1 if ag1 >= 65 else 0) + (1 if ag2 >= 65 else 0)
        fsi = fstat if y < (y_spouse_leaves_plan + 2) else 0 # Single after spouse leaves plan delayed 2 years
        dd['IRMAA-buk0'][y] = IRMAA_buk_n_size(dd['year'][y], 0, fsi, infl)
        dd['IRMAA-buk1'][y] = IRMAA_buk_n_size(dd['year'][y], 1, fsi, infl)
        dd['IRMAA-buk2'][y] = IRMAA_buk_n_size(dd['year'][y], 2, fsi, infl)
        dd['IRMAA-buk3'][y] = IRMAA_buk_n_size(dd['year'][y], 3, fsi, infl)
        dd['IRMAA-buk4'][y] = IRMAA_buk_n_size(dd['year'][y], 4, fsi, infl)
        dd['IRMAA-buk5'][y] = IRMAA_buk_n_size(dd['year'][y], 5, fsi, infl)
        dd['IRMAA-chg0'][y] = IRMAA_chg_n_size(dd['year'][y], 0, IRMAA_pax, infl)
        dd['IRMAA-chg1'][y] = IRMAA_chg_n_size(dd['year'][y], 1, IRMAA_pax, infl)
        dd['IRMAA-chg2'][y] = IRMAA_chg_n_size(dd['year'][y], 2, IRMAA_pax, infl)
        dd['IRMAA-chg3'][y] = IRMAA_chg_n_size(dd['year'][y], 3, IRMAA_pax, infl)
        dd['IRMAA-chg4'][y] = IRMAA_chg_n_size(dd['year'][y], 4, IRMAA_pax, infl)
        dd['IRMAA-chg5'][y] = IRMAA_chg_n_size(dd['year'][y], 5, IRMAA_pax, infl)
        # QCD
        n = 0 if dd['year'][y] < base_year_irs_brackets else dd['year'][y] - base_year_irs_brackets
        QCD_pax = (1 if ag1 >= 70 else 0) + (1 if ag2 >= 70 else 0)
        dd['QCD_limit'][y] = annual_QCD_limit_pp * ((1.0 + infl) ** n) * QCD_pax

    # Asset Allocation Glide Path
    if glide_box.value != 0:
        # linear interpolation: v = v0 + (y - y0) * (vn - v0)/(yn - y0)
        # y0 is 0, yn is years-1, y is y (below), v0 is fraZZ, vn is frhZZ
        yn = years - 1
        dba = frhba - fraba
        dsa = frhsa - frasa
        dbr = frhbr - frabr
        dsr = frhsr - frasr
        dbd = frhbd - frabd
        dsd = frhsd - frasd
        for y in range(1,years):
            dd['frac_bonds_a'][y] = (fraba + y * dba / yn) / 100
            dd['frac_stock_a'][y] = (frasa + y * dsa / yn) / 100
            dd['frac_bonds_r'][y] = (frabr + y * dbr / yn) / 100
            dd['frac_stock_r'][y] = (frasr + y * dsr / yn) / 100
            dd['frac_bonds_d'][y] = (frabd + y * dbd / yn) / 100
            dd['frac_stock_d'][y] = (frasd + y * dsd / yn) / 100

    # the historical_year_for_rates argument, if provided, overrides the user input
    #
    if historical_year_for_rates == None and hist_box.value != 'Use Values Below' and test_mode != '3-peat':
        historical_year_for_rates = int(hist_box.value)
    if historical_year_for_rates != None:
        # historical_year_for_rates is the plan base year + 1
        for year, row in hd[:].loc[historical_year_for_rates:historical_year_for_rates+len(dd['year'])-2].iterrows():
            i = year - historical_year_for_rates + 1
            dd['ror_stock'][i] = row['S']
            dd['ror_bonds'][i] = row['B']
            dd['dvd_stock'][i] = row['D']

    ##### TESTING! #####
    if test_mode == 'test_losses':
        for y in range(1,years,2):
            dd['ror_stock'][y] = -0.5 * dd['ror_stock'][y]
    
    return dd


# ##################### The MINLP Optimization ####################

VARS = [
    # Initial Values for year 0
    'afterTax',
    'e_Roth',
    'e_Taxd',
    'j_Roth',
    'j_Taxd',
    # AAA
    'aTax__cash',
    'aTax_bonds',
    'aTax_basis',
    'aTax_unrlz_gain',
    'aTax_unrlz_loss',
    # Configuration Values
    'disp_income',   #  year 0 (for option A & B); year 1..N for option A
    # LP Vars for years 1..N
    'discretionary_spend',
    'from_eRoth',
    'from_jRoth',
    'from_eTaxd',
    'from_jTaxd',
    'from_aTax',
    'to_aTax',
    'fm_aTax__cash',
    'fm_aTax_bonds',
    'fm_aTax_basis',
    'fm_aTax_unrlz',
    'to_aTax__cash',
    'to_aTax_bonds',
    'to_aTax_basis',
    'to_aTax_unrlz',
    'fm_aTax_unrlz_gain',
    'fm_aTax_unrlz_loss',
    'fm_aTax_frac',
    # Intermediate Calculated values
    'e_RMD',
    'j_RMD',
    'QCD',
    'nQCD',
    'auto_income', #  = e_RMD + j_RMD + SSA_income + misc_income + pension_income
    'e_RothConv',
    'j_RothConv',
    'taxable_income',
    'dividends',
    'capgains',
    'caplosss',
    'caplossz',
    'tax0',  # 0% income tax bucket
    'tax1',  # next (10%) income tax bucket, ...
    'tax2',
    'tax3',
    'tax4',
    'tax5',
    'tax6',
    'tax7',
    'taxb',  # OBBBA extra retirement deduction
    'MAGI',
    'IRMAA',
    'obbba_exc',
    'cgt0',  #  0% capital gains tax bucket
    'cgt15', # 15% capital gains tax bucket, ...
    'cgt20',
    #'ncgt0', # 0% offset capital gains tax bucket (filled with ordinary income), ...
    #'ncgt15',
    #'ncgt20',
    'ncgt', # not-capital-gains portion of highest income bucket 
    'income_tax',
    'net_pretax'
]

BINS = [
    'IRMAA-bin0', # IRMAA levels
    'IRMAA-bin1',
    'IRMAA-bin2',
    'IRMAA-bin3',
    'IRMAA-bin4',
    'IRMAA-bin5',
    'cgbin15', # taxable_income >= 15% capital gains bracket
    'cgbin20', # taxable_income >= 20% capital gains bracket
]

def lop_to_cents(x):
    """Truncate model (float) data to 5 decimal digits"""
    if x == None:
        return -0.0 # unique value to identify unconstrained/unused values
    else:
        return max(0,round(x, 3))

def lop_to_cents_signed(x):
    """Truncate model (float) data to 5 decimal digits"""
    if x == None:
        return -0.0 # unique value to identify unconstrained/unused values
    else:
        return round(x, 3)

def oorplp(dd, mode, objective, tout, glim):
    """Run OORPyLP with specified objective, 'net_pretax', 'net_postax', 
        or a non-string value for maximum DI with a specified residual
    """
    # mode: 0: default, 1: no capital losses 2: no capgains basis averaging 3: neither 4: full slow mode
    # the default mode 0 is the same as mode 3: neither capital losses nor cost basis averaging
    no_capgains_constraints = (mode == 0) or (mode == 2) or (mode == 3)
    no_capital_losses = (mode == 0) or (mode == 1) or (mode == 3)
    #
    #err_out.clear_output() # at start of each run
    drb_out.clear_output() # at start of each run
    # ? out_box.clear_output()
    # config values from UI
    def rori_r(y):
        return 1.0 + (dd['ror_stock'][y] * dd['frac_stock_r'][y] + dd['ror_bonds'][y] * dd['frac_bonds_r'][y])
    def rori_d(y):
        return 1.0 + (dd['ror_stock'][y] * dd['frac_stock_d'][y] + dd['ror_bonds'][y] * dd['frac_bonds_d'][y])
    # the model
    scip = pyscipopt.Model()
    # scip.setEmphasis(pyscipopt.SCIP_PARAMEMPHASIS.HARDLP) # ? NUMERICS, PHASEFEAS, CPSOLVER no help
    # set up problem
    YRS = len(dd['e']) - 1 # number of years of projection from base year 0
    IDX = range(0,YRS+1)   # 0 (base year) .. YRS (final year)
    ftab = dd['surplus'][YRS]
    vars = {}
    for v in VARS:
        vars[v] = {}
        for i in IDX:
            vars[v][i] = scip.addVar(vtype='C', name=f"Proje_{v}_{i}")
    for v in BINS:
        vars[v] = {}
        for i in IDX:
            vars[v][i] = scip.addVar(vtype='B', name=f"Proje_{v}_{i}")

    def MAGI_m2(y):
        # get MAGI for two years before y
        if y < 2:
            return get_nut(dd, 'MAGI_prebase') # MAGI for year before base year
        elif y == 2:
            return dd['MAGI'][0]    # MAGI for base year
        else:
            return vars['MAGI'][y-2]

    max_tab = 18316.2 # size of total account for "the 1%"
    # limit max_tab based on total starting account value
    bas_tab = dd['e_Roth'][0] + dd['j_Roth'][0] + dd['afterTax'][0] + dd['e_Taxd'][0] + dd['j_Taxd'][0]
    bas_tab = max(bas_tab, ftab)
    if bas_tab * 2.0 < max_tab:
        max_tab = bas_tab * 2.0
    max_xfr = max_tab / 6    # this is the max RMD at age 101 (was 3052.7)
    max_inc = max_xfr * 2.25 # fudge for adding in other income (was 6969.7)
    max_tax = 0.37 * max_inc # was 2187.2
    #display_warning(f'using upper bounds: {max_tab: 4.3f}, {max_xfr: 4.3f}, {max_inc: 4.3f}, {max_tax: 4.3f}')
    
    for y in range(1,YRS+1):
        infl = (1.0 + get_nut(dd, 'inflation')) ** y
        # Set bounds on variables for non-linear version
        scip.chgVarLb(vars['afterTax'][y], 0.000375) # less than $1
        scip.chgVarLb(vars['to_aTax_unrlz'][y], -max_xfr * infl)
        # Upper bounds
        scip.chgVarUb(vars[      'afterTax'][y], max_tab * infl)
        scip.chgVarUb(vars[    'aTax_basis'][y], max_tab * infl)
        scip.chgVarUb(vars['aTax_unrlz_gain'][y],max_tab * infl)
        scip.chgVarUb(vars['aTax_unrlz_loss'][y],max_tab * infl)
        scip.chgVarUb(vars[    'aTax_bonds'][y], max_tab * infl)
        scip.chgVarUb(vars[    'aTax__cash'][y], max_tab * infl)
        scip.chgVarUb(vars[        'e_Roth'][y], max_tab * infl)
        scip.chgVarUb(vars[        'e_Taxd'][y], max_tab * infl)
        scip.chgVarUb(vars[        'j_Roth'][y], max_tab * infl)
        scip.chgVarUb(vars[        'j_Taxd'][y], max_tab * infl)
        scip.chgVarUb(vars[    'net_pretax'][y], max_tab * infl)
        scip.chgVarUb(vars[    'e_RothConv'][y], max_tab * infl)
        scip.chgVarUb(vars[    'j_RothConv'][y], max_tab * infl)
        scip.chgVarUb(vars[         'e_RMD'][y], max_xfr * infl)
        scip.chgVarUb(vars[         'j_RMD'][y], max_xfr * infl)
        scip.chgVarUb(vars[    'from_eRoth'][y], max_xfr * infl)
        scip.chgVarUb(vars[    'from_jRoth'][y], max_xfr * infl)
        scip.chgVarUb(vars[    'from_eTaxd'][y], max_xfr * infl)
        scip.chgVarUb(vars[    'from_jTaxd'][y], max_xfr * infl)
        scip.chgVarUb(vars[     'from_aTax'][y], max_xfr * infl)
        scip.chgVarUb(vars[       'to_aTax'][y], max_xfr * infl)
        scip.chgVarUb(vars[ 'fm_aTax__cash'][y], max_xfr * infl)
        scip.chgVarUb(vars[ 'fm_aTax_bonds'][y], max_xfr * infl)
        scip.chgVarUb(vars[ 'fm_aTax_basis'][y], max_xfr * infl)
        scip.chgVarUb(vars[ 'fm_aTax_unrlz'][y], max_xfr * infl)
        scip.chgVarUb(vars[  'fm_aTax_frac'][y],     1.0)
        scip.chgVarUb(vars[ 'to_aTax__cash'][y], max_xfr * infl)
        scip.chgVarUb(vars[ 'to_aTax_bonds'][y], max_xfr * infl)
        scip.chgVarUb(vars[ 'to_aTax_basis'][y], max_xfr * infl)
        scip.chgVarUb(vars[ 'to_aTax_unrlz'][y], max_xfr * infl)
        scip.chgVarUb(vars[   'disp_income'][y], max_inc * infl)
        scip.chgVarUb(vars[   'auto_income'][y], max_inc * infl)
        scip.chgVarUb(vars['taxable_income'][y], max_inc * infl)
        scip.chgVarUb(vars[          'MAGI'][y], max_inc * infl)
        scip.chgVarUb(vars[      'capgains'][y], max_xfr * infl)
        scip.chgVarUb(vars[      'caplosss'][y], max_xfr * infl)
        scip.chgVarUb(vars[      'caplossz'][y], max_xfr * infl)
        scip.chgVarUb(vars[     'dividends'][y],   500.0 * infl)
        scip.chgVarUb(vars[    'income_tax'][y], max_tax * infl)
        scip.chgVarUb(vars[         'IRMAA'][y],    34.0 * infl)
        scip.chgVarUb(vars[          'taxb'][y],    20.0 * infl)
        scip.chgVarUb(vars[          'tax0'][y],    50.0 * infl)
        scip.chgVarUb(vars[          'tax1'][y],    30.0 * infl)
        scip.chgVarUb(vars[          'tax2'][y],    80.0 * infl)
        scip.chgVarUb(vars[          'tax3'][y],   120.0 * infl)
        scip.chgVarUb(vars[          'tax4'][y],   220.0 * infl)
        scip.chgVarUb(vars[          'tax5'][y],   120.0 * infl)
        scip.chgVarUb(vars[          'tax6'][y],   280.0 * infl)
        scip.chgVarUb(vars[          'tax7'][y], max_inc * infl)
        scip.chgVarUb(vars[     'obbba_exc'][y], max_inc * infl)
        scip.chgVarUb(vars[          'cgt0'][y],   140.0 * infl)
        scip.chgVarUb(vars[         'cgt15'][y],    80.0 * infl)
        scip.chgVarUb(vars[         'cgt20'][y], max_inc * infl)
        scip.chgVarUb(vars[          'ncgt'][y],   max_inc * infl)
        #scip.chgVarUb(vars[         'ncgt0'][y],   140.0 * infl)
        #scip.chgVarUb(vars[        'ncgt15'][y],    80.0 * infl)
        #scip.chgVarUb(vars[        'ncgt20'][y], max_inc * infl)
        scip.chgVarUb(vars[           'QCD'][y],   220.0 * infl)
        scip.chgVarUb(vars[          'nQCD'][y],   220.0 * infl)
        
    # Objective
    if isinstance(objective, str):
        scip.setObjective(vars[objective][YRS], sense="maximize")
        # subject to:
        for y in range(1,YRS+1):
            scip.addCons(dd['disp_income'][y] == vars['disp_income'][y])
    else:
        #scip.setObjective(vars['disp_income'][0], sense="maximize") # 'Maximize Spend'
        scip.setObjective(vars['discretionary_spend'][0], sense="maximize") # 'Maximize Spend'
        # subject to growth and minimum residual:
        scip.addCons(vars['net_pretax'][YRS] >= ftab) # 'Minimum Residual'
        # mantain spending curve
        for y in range(1,YRS+1):
            scip.addCons(vars['discretionary_spend'][y] == dd['spend_δ'][y] * vars['discretionary_spend'][y-1])
            scip.addCons(vars['disp_income'][y] == vars['discretionary_spend'][y] + dd['spend_essence'][y])
    
    # Initial Values Constraints
    scip.addCons(vars['e_Roth'][0] == dd['e_Roth'][0])
    scip.addCons(vars['e_Taxd'][0] == dd['e_Taxd'][0])
    scip.addCons(vars['j_Roth'][0] == dd['j_Roth'][0])
    scip.addCons(vars['j_Taxd'][0] == dd['j_Taxd'][0])
    scip.addCons(vars['afterTax'][0] == dd['afterTax'][0])
    scip.addCons(vars['aTax__cash'][0] == dd['aTax__cash'][0])
    scip.addCons(vars['aTax_bonds'][0] == dd['aTax_bonds'][0])
    scip.addCons(vars['aTax_basis'][0] == dd['aTax_basis'][0])
    if dd['aTax_unrlz'][0] < 0:
        scip.addCons(vars['aTax_unrlz_gain'][0] == 0)
        if no_capital_losses:
            scip.addCons(vars['aTax_unrlz_loss'][0] == 0) # make_planning_datadict will have issued a warning
        else:
            scip.addCons(vars['aTax_unrlz_loss'][0] == -dd['aTax_unrlz'][0])
    else:
        scip.addCons(vars['aTax_unrlz_loss'][0] == 0)
        scip.addCons(vars['aTax_unrlz_gain'][0] == dd['aTax_unrlz'][0])

    # Calculation Constraints
    for y in range(1,YRS+1):
        scip.addCons(vars['e_RMD'][y] == dd['e_RMD_factor'][y] * vars['e_Taxd'][y-1])
        scip.addCons(vars['j_RMD'][y] == dd['j_RMD_factor'][y] * vars['j_Taxd'][y-1])

        # AfterTax asset allocations
        scip.addCons(vars['afterTax'][y]  == vars['aTax__cash'][y] + vars['aTax_bonds'][y] + vars['aTax_basis'][y]  \
                                             + vars['aTax_unrlz_gain'][y] - vars['aTax_unrlz_loss'][y])
        scip.addCons(vars['from_aTax'][y] == vars['fm_aTax__cash'][y] + vars['fm_aTax_bonds'][y] \
                                             + vars['fm_aTax_basis'][y] \
                                             + vars['fm_aTax_unrlz_gain'][y] - vars['fm_aTax_unrlz_loss'][y])
        scip.addCons(vars['to_aTax'][y]   == vars['to_aTax__cash'][y] + vars['to_aTax_bonds'][y] \
                                             + vars['to_aTax_basis'][y] + vars['to_aTax_unrlz'][y])
        
        scip.addCons(vars['afterTax'][y] * dd['frac_stock_a'][y] \
                                 == vars['aTax_basis'][y] + vars['aTax_unrlz_gain'][y] - vars['aTax_unrlz_loss'][y])
        scip.addCons(vars['afterTax'][y] * dd['frac_bonds_a'][y] == vars['aTax_bonds'][y])
        
        scip.addCons(vars['fm_aTax__cash'][y] <= vars['aTax__cash'][y-1])
        scip.addCons(vars['fm_aTax_bonds'][y] <= vars['aTax_bonds'][y-1])

        if no_capgains_constraints:
            scip.addCons(vars['fm_aTax_basis'][y]      <= vars['aTax_basis'][y-1])
            scip.addCons(vars['fm_aTax_unrlz_gain'][y] <= vars['aTax_unrlz_gain'][y-1])
            scip.addCons(vars['fm_aTax_unrlz_loss'][y] <= vars['aTax_unrlz_loss'][y-1])
        else:
            scip.addCons(vars['fm_aTax_basis'][y]      == vars['fm_aTax_frac'][y] * vars['aTax_basis'][y-1])
            scip.addCons(vars['fm_aTax_unrlz_gain'][y] == vars['fm_aTax_frac'][y] * vars['aTax_unrlz_gain'][y-1])
            scip.addCons(vars['fm_aTax_unrlz_loss'][y] == vars['fm_aTax_frac'][y] * vars['aTax_unrlz_loss'][y-1])

        scip.addCons(vars['aTax_unrlz_gain'][y] - vars['aTax_unrlz_loss'][y] \
                         == (vars['aTax_unrlz_gain'][y-1] - vars['fm_aTax_unrlz_gain'][y]) \
                          - (vars['aTax_unrlz_loss'][y-1] - vars['fm_aTax_unrlz_loss'][y]) \
                          + vars['to_aTax_unrlz'][y])
            
        if no_capital_losses:
            scip.addCons(vars['aTax_unrlz_loss'][y] == 0)
        else:
            # unfortuately, the first form using a disjunction leads to symmetry problems in SCIP:
            #scip.addConsDisjunction([vars['aTax_unrlz_gain'][y] == 0, vars['aTax_unrlz_loss'][y] == 0])
            # so I use the multiplication instead:
            scip.addCons(vars['aTax_unrlz_gain'][y] * vars['aTax_unrlz_loss'][y] == 0)

        scip.addCons(vars['aTax__cash'][y] == vars['aTax__cash'][y-1] + vars['to_aTax__cash'][y] - vars['fm_aTax__cash'][y])
        scip.addCons(vars['aTax_bonds'][y] == vars['aTax_bonds'][y-1] + vars['to_aTax_bonds'][y] - vars['fm_aTax_bonds'][y])
        scip.addCons(vars['aTax_basis'][y] == vars['aTax_basis'][y-1] + vars['to_aTax_basis'][y] - vars['fm_aTax_basis'][y])
                
        scip.addCons(vars['to_aTax_unrlz'][y] == (dd['ror_stock'][y] - dd['dvd_stock'][y]) \
                                                     * (vars['aTax_basis'][y-1] \
                                                        + vars['aTax_unrlz_gain'][y-1]
                                                        - vars['aTax_unrlz_loss'][y-1]))
        
        scip.addCons(vars['dividends'][y] == dd['ror_bonds'][y] * vars['aTax_bonds'][y-1] \
                                             + dd['dvd_stock'][y] * (vars['aTax_basis'][y-1] \
                                                                    + vars['aTax_unrlz_gain'][y-1]
                                                                    - vars['aTax_unrlz_loss'][y-1]))
        
        scip.addCons(vars['capgains'][y] == vars['fm_aTax_unrlz_gain'][y])
        
        if no_capital_losses:
            scip.addCons(vars['caplosss'][y] == 0.0)
        else:
            scip.addCons(vars['caplosss'][y] + vars['caplossz'][y] == vars['fm_aTax_unrlz_loss'][y])
            scip.addCons(vars['caplosss'][y] <= 3.0)

        # Roth Conversions)
        scip.addCons(vars['e_RothConv'][y] <= vars['e_Taxd'][y-1])
        scip.addCons(vars['j_RothConv'][y] <= vars['j_Taxd'][y-1])
        rlim = get_nut(dd, 'Roth_conv_max')
        if rlim != 'unlimited':
            scip.addConsDisjunction([vars['e_RothConv'][y] + vars['j_RothConv'][y] == 0.0,
                                     # originally this was just: vars[rlim][y] == 0.0
                                     # but SCIP was too clever, and sometimes used the next highest tax bracket!
                                     #  ('Limit: 10% tax bracket', 'tax2'),
                                     #  ('Limit: 12% tax bracket', 'tax3'),
                                     #  ('Limit: 22% tax bracket', 'tax4'),
                                     # there must be a better way, but a working hack:
                                     vars['tax7'][y] == 0.0 if rlim == 'tax7' else
                                     vars['tax7'][y] + vars['tax6'][y] == 0.0 if rlim == 'tax6' else
                                     vars['tax7'][y] + vars['tax6'][y] + vars['tax5'][y] == 0.0 if rlim == 'tax5' else
                                     vars['tax7'][y] + vars['tax6'][y] + vars['tax5'][y] + vars['tax4'][y] == 0.0 \
                                        if rlim == 'tax4' else
                                     vars['tax7'][y] + vars['tax6'][y] + vars['tax5'][y] + vars['tax4'][y] \
                                         + vars['tax3'][y] + vars['cgbin15'][y] == 0.0 \
                                        if rlim == 'tax3' else
                                     vars['tax7'][y] + vars['tax6'][y] + vars['tax5'][y] + vars['tax4'][y] \
                                         + vars['tax3'][y] + vars['tax2'][y] + vars['cgt15'][y] == 0.0 \
                                        if rlim == 'tax2' else
                                     vars['tax7'][y] + vars['tax6'][y] + vars['tax5'][y] + vars['tax4'][y] \
                                         + vars['tax3'][y] + vars['tax2'][y] + vars['tax1'][y] \
                                         + vars['cgt15'][y] == 0.0 \
                                        if rlim == 'tax1' else
                                     vars['tax7'][y] + vars['tax6'][y] + vars['tax5'][y] + vars['tax4'][y] \
                                         + vars['tax3'][y] + vars['tax2'][y] + vars['tax1'][y] + vars['tax0'][y] \
                                         + vars['cgt15'][y] + vars['cgt0'][y] == 0.0 \
                                        if rlim == 'tax0' else
                                     vars[rlim][y] == 0.0
                                   ])

        # QCD Calculation
        scip.addCons(vars['QCD'][y] + vars['nQCD'][y] == dd['charity'][y])
        scip.addCons(vars['QCD'][y] <= dd['QCD_limit'][y])
        scip.addCons(vars['QCD'][y] <= vars['e_RMD'][y] + vars['j_RMD'][y] \
                                       + vars['from_eTaxd'][y] + vars['from_jTaxd'][y])

        # IRMAA Calculation
        # IRMAA-pax: 0, 1, 2 # individuals over 65
        # IRMAA-buk: income tier increments, 0..5
        # IRMAA-bin: binary indicator if tier is reached, 0..5
        # IRMAA-chg: (sur-)charge per tier, 0..5
        # MAGI <= IRMAA-bin[0] * IRMAA-buk[0] + ... IRMAA-bin[5] * IRMAA-buk[5]
        # IRMAA-bin[0] >= IRMAA-bin[1] >= ... IRMAA-bin[5] # fill the lower bins first
        # IRMAA = IRMAA-pax * (IRMAA-chg[n] * IRMAA-bin[n] for n in 0..5) 
        # dd has precomputed IRMAA-pax * IRMAA-chg
        
        scip.addCons(vars['IRMAA-bin0'][y] >= vars['IRMAA-bin1'][y])
        scip.addCons(vars['IRMAA-bin1'][y] >= vars['IRMAA-bin2'][y])
        scip.addCons(vars['IRMAA-bin2'][y] >= vars['IRMAA-bin3'][y])
        scip.addCons(vars['IRMAA-bin3'][y] >= vars['IRMAA-bin4'][y])
        scip.addCons(vars['IRMAA-bin4'][y] >= vars['IRMAA-bin5'][y])
        
        scip.addCons(MAGI_m2(y) <= vars['IRMAA-bin0'][y] * dd['IRMAA-buk0'][y] \
                                 + vars['IRMAA-bin1'][y] * dd['IRMAA-buk1'][y] \
                                 + vars['IRMAA-bin2'][y] * dd['IRMAA-buk2'][y] \
                                 + vars['IRMAA-bin3'][y] * dd['IRMAA-buk3'][y] \
                                 + vars['IRMAA-bin4'][y] * dd['IRMAA-buk4'][y] \
                                 + vars['IRMAA-bin5'][y] * dd['IRMAA-buk5'][y] \
                                 + 0.00001) # this <= $0.01 fudge factor seems to rescue some otherwise non-converging soltions

        scip.addCons(vars['IRMAA'][y] == vars['IRMAA-bin0'][y] * dd['IRMAA-chg0'][y] \
                                      + vars['IRMAA-bin1'][y] * dd['IRMAA-chg1'][y] \
                                      + vars['IRMAA-bin2'][y] * dd['IRMAA-chg2'][y] \
                                      + vars['IRMAA-bin3'][y] * dd['IRMAA-chg3'][y] \
                                      + vars['IRMAA-bin4'][y] * dd['IRMAA-chg4'][y] \
                                      + vars['IRMAA-bin5'][y] * dd['IRMAA-chg5'][y]) 

        # Income Calculation

        scip.addCons(vars['auto_income'][y] == vars['e_RMD'][y] + vars['j_RMD'][y] + vars['dividends'][y] \
                                                + dd['misc_income'][y] + dd['SSA_income'][y] \
                                                + dd['taxfree_income'][y] \
                                                + dd['pension_income'][y] - vars['IRMAA'][y])

        scip.addCons(vars['taxable_income'][y] == vars['e_RMD'][y] + vars['j_RMD'][y] + vars['dividends'][y] \
                                                + dd["misc_income"][y] + 0.85 * dd["SSA_income"][y] - vars['caplosss'][y] \
                                                + vars['from_eTaxd'][y] + vars['from_jTaxd'][y] - vars['QCD'][y] \
                                                + dd['pension_income'][y] + vars['e_RothConv'][y] + vars['j_RothConv'][y])

        # Spending
        
        scip.addCons(vars['disp_income'][y] == vars['auto_income'][y] + vars['from_aTax'][y] \
                                            + vars['from_eTaxd'][y] + vars['from_jTaxd'][y] \
                                            + vars['from_eRoth'][y] + vars['from_jRoth'][y] \
                                            - vars['income_tax'][y] - vars['to_aTax'][y])

        # Taxation - XXX qualified dividends?

        # OBBBA
        # MAGI used, so need to add the non-taxable portion of SSA
        # OBBBA-pax: 0, 1, 2 # individuals over 65
        # OBBBA_exc: == (MAGI - (OBBBA-pax * 75.000)
        # OBBBA-ded: <= OBBBA-pax * 6.000 - (OBBBA-pax * 0.06 * OBBBA_exc))

        scip.addCons(vars['MAGI'][y] == vars['e_RMD'][y] + vars['j_RMD'][y] + vars['dividends'][y] \
                                            + dd["misc_income"][y] + dd['pension_income'][y] + dd["SSA_income"][y] \
                                            + vars['from_eTaxd'][y] + vars['from_jTaxd'][y] - vars['caplosss'][y] \
                                            + vars['capgains'][y] + vars['e_RothConv'][y] + vars['j_RothConv'][y])
        
        scip.addCons(vars['obbba_exc'][y] >= vars['MAGI'][y] - dd['obbba_pax'][y] * 75.0) # lower bound is 0

        scip.addCons(vars['tax0'][y] <= dd['tax0'][y])
        scip.addCons(vars['tax1'][y] <= dd['tax1'][y])
        scip.addCons(vars['tax2'][y] <= dd['tax2'][y])
        scip.addCons(vars['tax3'][y] <= dd['tax3'][y])
        scip.addCons(vars['tax4'][y] <= dd['tax4'][y])
        scip.addCons(vars['tax5'][y] <= dd['tax5'][y])
        scip.addCons(vars['tax6'][y] <= dd['tax6'][y])
        scip.addCons(vars['taxb'][y] <= dd['obbba_pax'][y] * addl_obbba_deduction_age65 \
                                        - (dd['obbba_pax'][y] * 0.06 * vars['obbba_exc'][y]))

        scip.addCons(vars['taxable_income'][y] == vars['tax0'][y] \
                                                + vars['taxb'][y] \
                                                + vars['tax1'][y] \
                                                + vars['tax2'][y] \
                                                + vars['tax3'][y] \
                                                + vars['tax4'][y] \
                                                + vars['tax5'][y] \
                                                + vars['tax6'][y] \
                                                + vars['tax7'][y])
    
        # capgains tax
        
        scip.addCons(vars['capgains'][y] == vars['cgt0'][y] + vars['cgt15'][y] + vars['cgt20'][y])

        scip.addCons(vars['cgbin15'][y] >= vars['cgbin20'][y])

        scip.addCons(vars['taxable_income'][y] <= dd['cgt0'][y] + vars['taxb'][y] + 
                                                      vars['cgbin15'][y] * dd['cgt15'][y] + 
                                                      vars['cgbin20'][y] * 999.0)

        scip.addCons(vars['taxable_income'][y] == vars['cgbin15'][y] * (dd['cgt0'][y] + vars['taxb'][y]) +
                                                      vars['cgbin20'][y] * dd['cgt15'][y] + 
                                                      vars['ncgt'][y])

        scip.addCons(vars['cgt0'][y] <= (1 - vars['cgbin15'][y]) * (dd['cgt0'][y] + vars['taxb'][y] - vars['ncgt'][y]))

        scip.addCons(vars['cgt15'][y] <= (1 - vars['cgbin20'][y]) * (dd['cgt15'][y] - vars['ncgt'][y]))

        scip.addCons(vars['income_tax'][y] == 0.0 * vars['tax0'][y] \
                                            + 0.0 * vars['taxb'][y] \
                                            + 0.10 * vars['tax1'][y] \
                                            + 0.12 * vars['tax2'][y] \
                                            + 0.22 * vars['tax3'][y] \
                                            + 0.24 * vars['tax4'][y] \
                                            + 0.32 * vars['tax5'][y] \
                                            + 0.35 * vars['tax6'][y] \
                                            + 0.37 * vars['tax7'][y] \
                                            + 0.15 * vars['cgt15'][y] \
                                            + 0.20 * vars['cgt20'][y])

        # Annual Accounts Update
        
        scip.addCons(vars['e_Roth'][y] == rori_r(y) * (vars['e_Roth'][y-1] - vars['from_eRoth'][y] + vars['e_RothConv'][y]))
        scip.addCons(vars['e_Taxd'][y] == rori_d(y) * (vars['e_Taxd'][y-1] - vars['e_RMD'][y] \
                                         + dd['e_Taxd_in'][y] # lump sum pension distribution
                                         - vars['from_eTaxd'][y] - vars['e_RothConv'][y]))
        scip.addCons(vars['j_Roth'][y] == rori_r(y) * (vars['j_Roth'][y-1] - vars['from_jRoth'][y] + vars['j_RothConv'][y]))
        scip.addCons(vars['j_Taxd'][y] == rori_d(y) * (vars['j_Taxd'][y-1] - vars['j_RMD'][y] \
                                         + dd['j_Taxd_in'][y] # lump sum pension distribution
                                         - vars['from_jTaxd'][y] - vars['j_RothConv'][y]))

        # For "reasons," the optimizer moves all money into afterTax in the last year of the plan.
        # Until I figure out why, or discover a constraint to prevent that, the 0.999... hack...
        scip.addCons(vars['net_pretax'][y] \
                        == (0.999999 * vars['afterTax'][y] + vars['e_Taxd'][y] + vars['j_Taxd'][y] \
                            + vars['e_Roth'][y] + vars['j_Roth'][y]))

    scip.setParam("limits/time", tout)
    scip.setParam("limits/gap", glim / 100)
    scip.optimize()
    status = scip.getStatus()
    stage = scip.getStageName()
    
    # not effective: print('\n', flush = True)
    # not effective: sys.stdout.flush()
    time.sleep(1.0) # to flush output to the correct widget

    # with err_out:
    #     print(f'sta {status}')
    #     print(f'stg {stage}')

    if status == 'infeasible': # and stage == 'SOLVED':
        # fudge
        dd['net_pretax'][YRS] = 0
        dd['disp_income'][1] = 0

    else:
        
        #scip.writeProblem(filename="data/inital_model.mps", trans=False, genericnames=False)
        #scip.writeProblem(filename="data/xformd_model.mps", trans=True, genericnames=False)
        #scip.writeProblem(filename="data/inital_model.lp", trans=False, genericnames=False)
        #scip.writeProblem(filename="data/xformd_model.lp", trans=True, genericnames=False)
    
        OUTS = [
            'afterTax',
            'e_Roth',
            'e_Taxd',
            'j_Roth',
            'j_Taxd',
            'e_RMD',
            'j_RMD',
            'from_eRoth',
            'from_jRoth',
            'from_eTaxd',
            'from_jTaxd',
            'from_aTax',
            'to_aTax',
            'aTax__cash',
            'aTax_bonds',
            'aTax_basis',
            #'aTax_unrlz',
            'fm_aTax__cash',
            'fm_aTax_bonds',
            'fm_aTax_basis',
           #'fm_aTax_unrlz',
            'to_aTax__cash',
            'to_aTax_bonds',
            'to_aTax_basis',
           #'to_aTax_unrlz',
            'e_RothConv',
            'j_RothConv',
            'auto_income',
            'taxable_income',
            'dividends',
            'capgains',
            'caplosss',
            'disp_income',
            'income_tax',
            'QCD',
            'MAGI',
            'IRMAA',
            'net_pretax']
    
        for n in OUTS:
            for y in range(1,YRS+1):
                v = scip.getVal(vars[n][y])
                dd[n][y] = lop_to_cents(v)
    
        dd['disp_income'][0] = lop_to_cents(scip.getVal(vars['disp_income'][0])) # set if maximzing spend
        
        for y in range(1,YRS+1):
            dd['aTax_unrlz'][y] = lop_to_cents_signed(scip.getVal(vars['aTax_unrlz_gain'][y]) \
                                               - scip.getVal(vars['aTax_unrlz_loss'][y]))
            dd['fm_aTax_unrlz'][y] = lop_to_cents_signed(scip.getVal(vars['fm_aTax_unrlz_gain'][y]) \
                                                  - scip.getVal(vars['fm_aTax_unrlz_loss'][y]))
            dd['to_aTax_unrlz'][y] = lop_to_cents_signed(scip.getVal(vars['to_aTax_unrlz'][y]))
            
            dd['tax_bracket'][y] = \
                0.32 if lop_to_cents(scip.getVal(vars['tax5'][y])) != 0 else \
                0.24 if lop_to_cents(scip.getVal(vars['tax4'][y])) != 0 else \
                0.22 if lop_to_cents(scip.getVal(vars['tax3'][y])) != 0 else \
                0.12 if lop_to_cents(scip.getVal(vars['tax2'][y])) != 0 else \
                0.10 if lop_to_cents(scip.getVal(vars['tax1'][y])) != 0 else \
                0.00
            dd['cgains_rate'][y] = \
                0.20 if lop_to_cents(scip.getVal(vars['cgt20'][y])) != 0 else \
                0.15 if lop_to_cents(scip.getVal(vars['cgt15'][y])) != 0 else \
                0.00
            #
            dd['IRMAA-bins'][y] = scip.getVal(vars['IRMAA-bin0'][y]) \
                                + scip.getVal(vars['IRMAA-bin1'][y]) * 2 \
                                + scip.getVal(vars['IRMAA-bin2'][y]) * 4 \
                                + scip.getVal(vars['IRMAA-bin3'][y]) * 8 \
                                + scip.getVal(vars['IRMAA-bin4'][y]) * 16 \
                                + scip.getVal(vars['IRMAA-bin5'][y]) * 32
            #
            dd['net_postax'][y] = dd['e_Roth'][y] + dd['j_Roth'][y] \
                                    + (1.0 - dd['cgains_rate'][y]) * dd['afterTax'][y] \
                                    + (1.0 - dd['tax_bracket'][y]) * (dd['e_Taxd'][y] + dd['j_Taxd'][y])
    
        # with err_out:
        #     for y in range(1,3):
        #         print(f"00 {lop_to_cents(scip.getVal(vars['cgt0'][y]))}")
        #         print(f"15 {lop_to_cents(scip.getVal(vars['cgt15'][y]))}")
        #         print(f"20 {lop_to_cents(scip.getVal(vars['cgt20'][y]))}")
        #         print(f"b  {lop_to_cents(scip.getVal(vars['taxb'][y]))}")
        #         print(f"nc {lop_to_cents(scip.getVal(vars['ncgt'][y]))}")
        #         print(f"bn {scip.getVal(vars['cgbin15'][y] + 2 * scip.getVal(vars['cgbin20'][y]))}")
                
        # with err_out:
            # print(f'obj {scip.getObjVal()}')
            # print(f'sec {scip.getSolvingTime()}')

    gap = scip.getGap()
    stime = scip.getSolvingTime()
    
    scip.freeProb() # removes model from its cache memory
    
    return (status, dd['net_pretax'][YRS], dd['disp_income'][1], stage, gap, stime)
    

######################## Controls and Outputs ############################

# The big general purpose output for tables, graphs, etc.
out_box = widgets.Output(layout={'border': '1px solid black'})

def display_dd(dd, fname=None):
    with out_box:
        dfr = pd.DataFrame(dd, index = dd["year"])
        if fname != None:
            dfr.to_csv(fname)
        df = pd.DataFrame(dfr[1:])
        # barmode='relative' is same as barmode='stack' except that negative values plot below the x-axis
        display(px.bar(df, barmode='relative', x='year',
                    y=['afterTax', 'e_Taxd', 'j_Taxd', 'e_Roth', 'j_Roth'],
                    color_discrete_map={'afterTax':'royalblue', 'e_Taxd':'mediumorchid', 'j_Taxd':'mediumpurple',
                                        'e_Roth':'forestgreen', 'j_Roth':'lawngreen'},
                    title='Nominal Balances'))
        df['to_aTax'] = -df['to_aTax']
        df['net_aTax'] = df[['from_aTax', 'to_aTax']].sum(axis=1)
        display(px.bar(df, barmode='relative', x='year', # indianred or firebrick for brick?
                    y=['SSA_income','pension_income','e_RMD','j_RMD',
                       'from_eTaxd','from_jTaxd','net_aTax','from_eRoth','from_jRoth'], # 'from_aTax','to_aTax'
                    color_discrete_map={'SSA_income':'goldenrod','pension_income':'darkgoldenrod', # 'to_aTax':'royalblue',
                                        'e_RMD':'firebrick','j_RMD':'chocolate','to_eTaxd':'mediumorchid',
                                        'from_jTaxd':'mediumpurple','net_aTax':'royalblue', 'from_eTaxd':'mediumorchid',
                                        'from_eRoth':'forestgreen','from_jRoth':'lawngreen'}, # 'from_aTax':'royalblue',
                    title='Nominal Withdrawals'))
        # combine (from_eRoth + from_jRoth), (e_RMD + j_RMD + from_eTaxd + from_jTaxd)
        df['from_Roth'] = df[['from_eRoth', 'from_jRoth']].sum(axis=1)
        df['from_Taxd'] = df[['e_RMD', 'j_RMD', 'from_eTaxd', 'from_jTaxd']].sum(axis=1)
        df['guaranteed'] = df[['SSA_income', 'pension_income']].sum(axis=1)
        display(px.bar(df, barmode='relative', x='year',
                    y=['guaranteed', 'net_aTax', 'from_Roth', 'from_Taxd'], # 'from_aTax', 'to_aTax'
                    color_discrete_map={'guaranteed':'goldenrod','net_aTax':'royalblue', # 'from_aTax':'royalblue','to_aTax':'royalblue',
                                        'from_Roth':'forestgreen','from_Taxd':'mediumorchid'},
                    title='Nominal Withdrawals'))
        # Tax Data
        df['income_tax'] = -df['income_tax']
        df['Roth_conv'] = df[['e_RothConv', 'j_RothConv']].sum(axis=1)
        df['wthd_Taxd'] = df[['from_eTaxd', 'from_jTaxd']].sum(axis=1)
        df['total_RMD'] = df[['e_RMD', 'j_RMD']].sum(axis=1)
        display(px.bar(df, barmode='relative', x='year',
                       y=['income_tax', 'Roth_conv', 'wthd_Taxd', 'total_RMD', 'SSA_income', 'pension_income', 'dividends'],
                       color_discrete_map={'income_tax':'firebrick', 'Roth_conv':'forestgreen',
                                           'wthd_Taxd':'mediumorchid', 'total_RMD':'chocolate',
                                           'SSA_income':'goldenrod', 'pension_income':'darkgoldenrod',
                                           'dividends':'royalblue'},
                       title='Tax Data'))
        # Tables
        # Nominal Balances
        display(Markdown('\n### Nominal Balances'))
        display(pd.DataFrame(df, columns=['e', 'j', 'afterTax', 'aTax_basis', 'e_Roth', 'j_Roth',
                                           'e_Taxd', 'j_Taxd', 'net_pretax', 'net_postax']))
        # Nominal Withdrawals
        display(Markdown('\n### Nominal Withdrawals'))
        display(pd.DataFrame(df, columns=['e', 'j', 'e_RMD', 'j_RMD', 'from_eTaxd', 'from_jTaxd',
                                          'from_aTax', 'to_aTax', 'from_eRoth', 'from_jRoth']))
        # Spend info
        # 0 = 
        # # income
        # + dd['e_RMD'][y]
        # + dd['j_RMD'][y]
        # + dd['SSA_income'][y]
        # + dd['misc_income'][y]
        # + dd['pension_income'][y]
        # + dd['taxfree_income'][y]
        # + dd['dividends'][y]
        # # withdrawls
        # + dd['from_aTax'][y]
        # + dd['from_eTaxd'][y]
        # + dd['from_jTaxd'][y]
        # + dd['from_eRoth'][y]
        # + dd['from_jRoth'][y]
        # + dd['e_RothConv'][y]
        # + dd['j_RothConv'][y]
        # # transfers
        # - dd['to_aTax'][y]
        # - dd['e_RothConv'][y]
        # - dd['j_RothConv'][y]
        # # spending
        # - dd['IRMAA'][y]
        # - dd['income_tax'][y]
        # - dd['disp_income'][y]
        df['IRMAA'] = -df['IRMAA']
        df['fixed_income'] = df[['total_RMD', 'SSA_income', 'pension_income', 'taxfree_income',
                                 'misc_income', 'dividends']].sum(axis=1)
        df['withdrawals'] = df[['wthd_Taxd', 'from_Roth', 'Roth_conv', 'from_aTax']].sum(axis=1)
        df['nRoth_conv'] = -df['Roth_conv']
        df['transfers'] = df[['nRoth_conv', 'to_aTax']].sum(axis=1)
        display(Markdown('\n### Nominal Fixed Income'))
        display(pd.DataFrame(df, columns=['e', 'j', 'total_RMD', 'SSA_income', 'pension_income', 'misc_income', 'dividends']))
        df['DI'] = df['disp_income']
        display(Markdown('\n### Nominal Spending'))
        display(pd.DataFrame(df, columns=['e', 'j', 'fixed_income', 'withdrawals', 'transfers', 'IRMAA-bins', 'IRMAA', 'income_tax', 'DI']))
        # Tax info
        display(Markdown('\n### Tax Info'))
        df['caplosss'] = -df['caplosss']
        display(pd.DataFrame(df, columns=['Roth_conv', 'wthd_Taxd', 'fixed_income', 'capgains', 'caplosss', 'QCD',
                                          'taxable_income', 'income_tax', 'tax_bracket', 'cgains_rate', 'MAGI']))
        # After Tax
        display(Markdown('\n### After Tax Account Details'))
        df['fm_aTax__cash'] = -df['fm_aTax__cash']
        df['fm_aTax_bonds'] = -df['fm_aTax_bonds']
        df['fm_aTax_basis'] = -df['fm_aTax_basis']
        df['fm_aTax_unrlz'] = -df['fm_aTax_unrlz']
        df['net_aTax__cash'] = df[['fm_aTax__cash', 'to_aTax__cash']].sum(axis=1)
        df['net_aTax_bonds'] = df[['fm_aTax_bonds', 'to_aTax_bonds']].sum(axis=1)
        df['net_aTax_basis'] = df[['fm_aTax_basis', 'to_aTax_basis']].sum(axis=1)
        df['net_aTax_unrlz'] = df[['fm_aTax_unrlz', 'to_aTax_unrlz']].sum(axis=1)
        display(pd.DataFrame(df, columns=['net_aTax__cash', 'net_aTax_bonds', 'net_aTax_basis', 'net_aTax_unrlz',
                                          'aTax__cash', 'aTax_bonds', 'aTax_basis', 'aTax_unrlz']))
        # testing...
        # def spnd(age1, age2, targetspend):
        #     # real_delta = 0.00008 * (age1 * age2) - (0.0125 * (age1 + age2) / 2) - 0.0066 * ln(targetspend) + 54.6%
        #     return 0.00008 * (age1 * age2) - (0.0125 * (age1 + age2) / 2) - 0.0066 * math.log(targetspend*1000) + 0.546
        # df['real spend δ'] = df.apply(lambda x: spnd(x.e, x.j, x.'disp_income'), axis=1)
        
        df['real spend δ'] = df.apply(lambda x: (x.spend_δ / ((1.0 + get_nut(dd, 'inflation')) ** (x.year - dd['year'][0])) - 1.0), axis=1)
        df['nominal spend δ'] = df['spend_δ'] - 1.0
        #display(px.line(df, x='year', y=['real spend δ', 'nominal spend δ'], title='spend curve real and nominal'))
        display(px.line(df, x='year', y='nominal spend δ', title='nominal spend curve'))

        df['real DI'] = df.apply(lambda x: x.disp_income / ((1.0 + get_nut(dd, 'inflation')) ** (x.year - dd['year'][0])), axis=1)
        display(px.line(df, x='year', y=['real DI', 'DI'], title='real and nominal spending'))

        ### Real value reports
        # Real Balances
        df['afterTax'] = df.apply(lambda x: x.afterTax / ((1.0 + get_nut(dd, 'inflation')) ** (x.year - dd['year'][0])), axis=1)
        df['e_Taxd'] = df.apply(lambda x: x.e_Taxd / ((1.0 + get_nut(dd, 'inflation')) ** (x.year - dd['year'][0])), axis=1)
        df['j_Taxd'] = df.apply(lambda x: x.j_Taxd / ((1.0 + get_nut(dd, 'inflation')) ** (x.year - dd['year'][0])), axis=1)
        df['e_Roth'] = df.apply(lambda x: x.e_Roth / ((1.0 + get_nut(dd, 'inflation')) ** (x.year - dd['year'][0])), axis=1)
        df['j_Roth'] = df.apply(lambda x: x.j_Roth / ((1.0 + get_nut(dd, 'inflation')) ** (x.year - dd['year'][0])), axis=1)
        display(px.bar(df, barmode='relative', x='year',
                    y=['afterTax', 'e_Taxd', 'j_Taxd', 'e_Roth', 'j_Roth'],
                    color_discrete_map={'afterTax':'royalblue', 'e_Taxd':'mediumorchid', 'j_Taxd':'mediumpurple',
                                        'e_Roth':'forestgreen', 'j_Roth':'lawngreen'},
                    title='Real Balances (Base Year $000s)'))
        # Real Withdrawals
        df['guaranteed'] = df.apply(lambda x: x.guaranteed / ((1.0 + get_nut(dd, 'inflation')) ** (x.year - dd['year'][0])), axis=1)
        df['net_aTax'] = df.apply(lambda x: x.net_aTax / ((1.0 + get_nut(dd, 'inflation')) ** (x.year - dd['year'][0])), axis=1)
        df['from_Roth'] = df.apply(lambda x: x.from_Roth / ((1.0 + get_nut(dd, 'inflation')) ** (x.year - dd['year'][0])), axis=1)
        df['from_Taxd'] = df.apply(lambda x: x.from_Taxd / ((1.0 + get_nut(dd, 'inflation')) ** (x.year - dd['year'][0])), axis=1)
        display(px.bar(df, barmode='relative', x='year',
                    y=['guaranteed', 'net_aTax', 'from_Roth', 'from_Taxd'], # 'from_aTax', 'to_aTax'
                    color_discrete_map={'guaranteed':'goldenrod','net_aTax':'royalblue', # 'from_aTax':'royalblue','to_aTax':'royalblue',
                                        'from_Roth':'forestgreen','from_Taxd':'mediumorchid'},
                    title='Real Withdrawals (Base Year $000s)'))
        # Real Balances
        df['aTax_basis'] = df.apply(lambda x: x.aTax_basis / ((1.0 + get_nut(dd, 'inflation')) ** (x.year - dd['year'][0])), axis=1)
        df['net_pretax'] = df.apply(lambda x: x.net_pretax / ((1.0 + get_nut(dd, 'inflation')) ** (x.year - dd['year'][0])), axis=1)
        df['net_postax'] = df.apply(lambda x: x.net_postax / ((1.0 + get_nut(dd, 'inflation')) ** (x.year - dd['year'][0])), axis=1)
        display(Markdown('\n### Real Balances (Base Year $000s)'))
        display(pd.DataFrame(df, columns=['e', 'j', 'afterTax', 'aTax_basis', 'e_Roth', 'j_Roth',
                                           'e_Taxd', 'j_Taxd', 'net_pretax', 'net_postax']))
        # Real Withdrawals
        df['e_RMD'] = df.apply(lambda x: x.e_RMD / ((1.0 + get_nut(dd, 'inflation')) ** (x.year - dd['year'][0])), axis=1)
        df['j_RMD'] = df.apply(lambda x: x.j_RMD / ((1.0 + get_nut(dd, 'inflation')) ** (x.year - dd['year'][0])), axis=1)
        df['to_aTax'] = df.apply(lambda x: x.to_aTax / ((1.0 + get_nut(dd, 'inflation')) ** (x.year - dd['year'][0])), axis=1)
        df['from_aTax'] = df.apply(lambda x: x.from_aTax / ((1.0 + get_nut(dd, 'inflation')) ** (x.year - dd['year'][0])), axis=1)
        display(Markdown('\n### Real Withdrawals (Base Year $000s)'))
        display(pd.DataFrame(df, columns=['e', 'j', 'e_RMD', 'j_RMD', 'from_eTaxd', 'from_jTaxd',
                                          'from_aTax', 'to_aTax', 'from_eRoth', 'from_jRoth']))
        # Returns
        display(px.bar(df, barmode='group', x='year',
                    y=['ror_stock', 'dvd_stock', 'ror_bonds'],
                    color_discrete_map={'ror_stock':'mediumorchid', 'dvd_stock':'goldenrod',
                                        'ror_bonds':'forestgreen'},
                    title='Rates of Return'))


def oorp(mode, objt, test, tout, glim, fname=None):
    """Run the projection based on widget inputs and display results"""
    # mode: 0: default, 1: no capital losses 2: no capgains basis averaging 3: neither 4: full minlp
    # objt: objective
    # test: a value for development testing
    # tout: timeout in seconds for solver
    # glim: gap limit for solver
    err_out.clear_output() # at start of each run
    out_box.clear_output()
    dd = make_planning_datadict(test)
    #
    set_nut(dd, 'orp_mode', mode)
    set_nut(dd, 'orp_objtv', objt)
    set_nut(dd, 'time_limit', tout)
    set_nut(dd, 'gap_limit', glim)
    #
    # the projection...
    with drb_out:
        (status, net_pretax, di, stage, gap, stime) = oorplp(dd, mode, objt, tout, glim)
        print('\n', flush = True)
    #
    set_nut(dd, 'scip_status', status)
    set_nut(dd, 'scip_stage', stage)
    set_nut(dd, 'scip_gap', gap)
    set_nut(dd, 'scip_time', stime)
    #
    with out_box:
        if status == 'timelimit' or status == 'optimal' or status == 'gaplimit':
            print(f'FTAB: {net_pretax: 4.3f} DI 1st year: {di: 3.3f} Status: {status} at stage {stage} gap {100 * gap: 1.3f}% in {stime: 2.3f} s')
            # estimated post-tax: {net_postax: 7.3f}
        else:
            print(f'Optimization failed, status {status} gap {100 * gap: 1.3f}% in {stime: 2.3f} s')
        display_dd(dd, fname)

def walk_lap(fromyear, mode, tout, glim):
    # mode: 0: default, 1: no capital losses 2: no capgains basis averaging 3: neither 4: full minlp
    # objt: objective
    # test: a value for development testing
    # tout: timeout in seconds for solver
    # glim: gap limit for solver
    # mode = 2 # allow losses, no basis rationalization
    objt = 0 # max spend
    test = '' # no testing
    #
    dd = make_planning_datadict(test, fromyear)
    #
    set_nut(dd, 'orp_mode', mode)
    set_nut(dd, 'orp_objtv', objt)
    set_nut(dd, 'time_limit', tout)
    set_nut(dd, 'gap_limit', glim)
    #
    # the random data
    # 
    # -- these values are not independent!
    #  mu, sigma = 0.086, 0.182 # mean and standard deviation of s&p500 returns 1872 to 2024
    #  s = np.random.normal(mu, sigma, len(dd['year']))
    #  for y, x in enumerate(s):
    #      dd['ror_stock'][y] = x
    #  mu, sigma = 0.05885, 0.030848 # mean and standard deviation of 10-year Treasury rates 1970 to 2025
    #  s = np.random.normal(mu, sigma, len(dd['year']))
    #  for y, x in enumerate(s):
    #      dd['ror_bonds'][y] = max(0,x) # no negative bond yields
    #  mu, sigma = 0.02766, 0.01262 # mean and standard deviation of s&p500 dividend rates 1970 to 2025
    #  s = np.random.normal(mu, sigma, len(dd['year']))
    #  for y, x in enumerate(s):
    #      dd['dvd_stock'][y] = max(0,x) # no negative dividends
    #
    # the projection...
    with drb_out:
        (status, net_pretax, di, stage, gap, stime) = oorplp(dd, mode, objt, tout, glim)
        print('\n', flush = True)
    #
    set_nut(dd, 'scip_status', status)
    set_nut(dd, 'scip_stage', stage)
    set_nut(dd, 'scip_gap', gap)
    set_nut(dd, 'scip_time', stime)
    #
    return (dd, status, net_pretax, di, stage, gap, stime)

def walk(mode, tout, glim, fname):
    err_out.clear_output() # at start of each run
    out_box.clear_output()
    #
    worst_di = 1000000.0 # big enough to ensure the first solution will be the initial worst year
    #
    sta_yr = 1970
    nyears = 10 # TODO: configurable?
    if hist_box.value != 'Use Values Below':
        sta_yr = int(hist_box.value)
    #
    for y in range(sta_yr, sta_yr + nyears):
        (dd, status, net_pretax, di, stage, gap, stime) = walk_lap(y, mode, tout, glim)
        #
        with out_box:
            if status == 'timelimit' or status == 'optimal' or status == 'gaplimit':
                print(f'{y} | DI 1st year: {di: 3.3f} Status: {status} at stage {stage} gap {100 * gap: 1.3f}% in {stime: 2.3f} s')
                if dd['disp_income'][1] < worst_di:
                    worst_di = dd['disp_income'][1]
                    worst_di_dd = dd
                    worst_year = y
            else:
                print(f'{y} | Optimization failed, status {status} gap {100 * gap: 1.3f}% in {stime: 2.3f} s')
    with out_box:
        print(f'{worst_year} | DI 1st year: {worst_di: 3.3f} is lowest...')
    display_dd(worst_di_dd, fname) # 'data/_worst.csv'

def three_peat(mode, tout, glim, fname):
    err_out.clear_output() # at start of each run
    out_box.clear_output()
    #
    verbose = True
    objt = 0 # default, max DI
    #
    dd = make_planning_datadict('3-peat')
    #
    set_nut(dd, 'orp_mode', mode)
    set_nut(dd, 'orp_objtv', objt)
    set_nut(dd, 'time_limit', tout)
    set_nut(dd, 'gap_limit', glim)
    #
    def fnvers(i):
        p = fname.split('.')
        e = p.pop() # 'csv'
        p.append(str(i))
        p.append(e)
        return '.'.join(p)
    #
    historical_year_for_rates = 1970
    if hist_box.value != 'Use Values Below':
        historical_year_for_rates = int(hist_box.value)
    #
    # i-ORP: left five(!?) columns of the report are Shiller's historical data
    # i-ORP input instructions mention: S&P 500 index, dividends, bond returns and inflation
    # year, maybe?
    # the report (all real values): year, age, DI, TAB start of year, withdrawals, Roth_conv, income_taxes
    # the summary (all real values): initial DI, smallest DI, mean DI, stddev DI, FTAB, Total (sum of) DI
    rs = {
        'year':        list(dd['year']),
        'e':           list(dd['e']),
        'j':           list(dd['j']),
        # outputs...
        'hist_year':   list(dd['year']),
        'ror_stock':   dd['ror_stock'].copy(),
        'dvd_stock':   dd['dvd_stock'].copy(),
        'ror_bonds':   dd['ror_bonds'].copy(),
        'infl_rate':   dd['spend_δ'].copy(),
        'net_pretax':  dd['net_pretax'].copy(),
        'disp_income': dd['disp_income'].copy(),
        'IRMAA':       dd['IRMAA'].copy(),
        'income_tax':  dd['income_tax'].copy(),
        'withdrawals': dd['from_aTax'].copy(),
        'x_RothConv':  dd['e_RothConv'].copy(),
    }
    sm = { 'initial_DI': 0, 'smallest_DI': 0, 'mean_DI': 0, 'sdtdev_DI': 0, 'FTAB': 0, 'sum_DI': 0 }
    #
    cumm_infl = 1.0 # inflation relative to base year
    # TAB at start of year
    rs['net_pretax'][1] = dd['afterTax'][0] + dd['e_Taxd'][0] + dd['j_Taxd'][0] + dd['e_Roth'][0] + dd['j_Roth'][0]
    #
    i = 0
    for year, row in hd[:].loc[historical_year_for_rates:historical_year_for_rates+len(dd['year'])-2].iterrows():
        # the projection...
        with drb_out:
            (status, net_pretax, di, stage, gap, stime) = oorplp(dd, mode, objt, tout, glim)
            print('\n', flush = True)
        #
        set_nut(dd, 'scip_status', status)
        set_nut(dd, 'scip_stage', stage)
        set_nut(dd, 'scip_gap', gap)
        set_nut(dd, 'scip_time', stime)
        #
        with out_box:
            if status == 'timelimit' or status == 'optimal' or status == 'gaplimit':
                print(f'FTAB: {net_pretax: 4.3f} DI 1st year: {di: 3.3f} Status: {status} at stage {stage} gap {100 * gap: 1.3f}% in {stime: 2.3f} s')
            else:
                print(f'Optimization failed, status {status} gap {100 * gap: 1.3f}% in {stime: 2.3f} s')
            # display_dd(dd, fname)
        #
        i = year - historical_year_for_rates + 1
        #
        if verbose:
            dfr = pd.DataFrame(dd, index = dd["year"])
            dfr.to_csv(fnvers(i))
        #
        rs['hist_year'][i] = year
        rs['ror_stock'][i] = ror_stock = row['S']
        rs['ror_bonds'][i] = ror_bonds = row['B']
        rs['dvd_stock'][i] = dvd_stock = row['D']
        rs['infl_rate'][i] = infl_rate = row['I']
        #
        def rori_r(y):
            return 1.0 + (ror_stock * dd['frac_stock_r'][y] + ror_bonds * dd['frac_bonds_r'][y])
        def rori_d(y):
            return 1.0 + (ror_stock * dd['frac_stock_d'][y] + ror_bonds * dd['frac_bonds_d'][y])
        # adjust account values for rates
        dd['e_Taxd'][1] = rori_d(1) * (
                             dd['e_Taxd'][0]
                           - dd['e_RMD'][1]
                           - dd['e_RothConv'][1]
                           - dd['from_eTaxd'][1]
                           + dd['e_Taxd_in'][1]) # lump sum pension distribution
        dd['j_Taxd'][1] = rori_d(1) * (
                             dd['j_Taxd'][0]
                           - dd['j_RMD'][1]
                           - dd['j_RothConv'][1]
                           - dd['from_jTaxd'][1]
                           + dd['j_Taxd_in'][1]) # lump sum pension distribution
        dd['e_Roth'][1] = rori_r(1) * (dd['e_Roth'][0] - dd['from_eRoth'][1] + dd['e_RothConv'][1])
        dd['j_Roth'][1] = rori_r(1) * (dd['j_Roth'][0] - dd['from_jRoth'][1] + dd['j_RothConv'][1])
        #
        #chk_re(107, dd['aTax_unrlz'][y], dd['aTax_unrlz'][y-1] + dd['to_aTax_unrlz'][y] - dd['fm_aTax_unrlz'][y])
        #chk_re(108, dd['aTax_basis'][y], dd['aTax_basis'][y-1] + dd['to_aTax_basis'][y] - dd['fm_aTax_basis'][y])
        #chk_re(109, dd['aTax_bonds'][y], dd['aTax_bonds'][y-1] + dd['to_aTax_bonds'][y] - dd['fm_aTax_bonds'][y])
        #chk_re(110, dd['aTax__cash'][y], dd['aTax__cash'][y-1] + dd['to_aTax__cash'][y] - dd['fm_aTax__cash'][y])
        #chk_re(111, dd['to_aTax_unrlz'][y], (dd['ror_stock'][y] - dd['dvd_stock'][y]) 
        #                                       * (dd['aTax_basis'][y-1] + dd['aTax_unrlz'][y-1]))
        assumed_div = dd['dividends'][1]
        actual_divd = ror_bonds * dd['aTax_bonds'][0] + dvd_stock * (dd['aTax_basis'][0] + dd['aTax_unrlz'][0])
        if actual_divd < assumed_div:
            if dd['aTax__cash'][1] > (assumed_div - actual_divd):
                dd['aTax__cash'][1] -= (assumed_div - actual_divd)
                dd['afterTax'][1] -= (assumed_div - actual_divd)
            else:
                dd['disp_income'][1] -= (assumed_div - actual_divd)
        else:
                dd['aTax__cash'][1] -= (assumed_div - actual_divd)
                dd['afterTax'][1] -= (assumed_div - actual_divd)
        # infl
        cumm_infl *= (1.0 + infl_rate)
        infl_adj = 1.0 + (infl_rate - get_nut(dd, 'inflation'))
        #
        rs['disp_income'][i] = dd['disp_income'][1] / cumm_infl
        rs['withdrawals'][i] = (dd['e_RMD'][1] + dd['j_RMD'][1] + 
                                dd['from_eTaxd'][1] + dd['from_jTaxd'][1] +
                                dd['from_aTax'][1] - dd['to_aTax'][1] + 
                                dd['from_eRoth'][1] + dd['from_jRoth'][1])  / cumm_infl
        rs['x_RothConv'][i] = (dd['e_RothConv'][1] + dd['j_RothConv'][1]) / cumm_infl
        rs['income_tax'][i] = dd['income_tax'][1] / cumm_infl
        rs['IRMAA'][i] = dd['IRMAA'][1] / cumm_infl
        # 
        rs['net_pretax'][i+1] = (dd['afterTax'][1] + dd['e_Taxd'][1] + dd['j_Taxd'][1] + 
                                 dd['e_Roth'][1] + dd['j_Roth'][1]) / cumm_infl
        #
        if len(dd['e']) < 5:
            break
        #
        # adjust dd for advancing year
        for _,n in squirrel_map.items():
            dd[n][1] = dd[n][0]
        # infl
        for y in range(2,len(dd['e'])):
            # TODO -- not yet handled: pension COLAs, phases of retirement event adjustments
            dd['QCD_limit'][y] *= infl_adj
            dd['IRMAA-buk0'][y] *= infl_adj
            dd['IRMAA-buk1'][y] *= infl_adj
            dd['IRMAA-buk2'][y] *= infl_adj
            dd['IRMAA-buk3'][y] *= infl_adj
            dd['IRMAA-buk4'][y] *= infl_adj
            dd['IRMAA-buk5'][y] *= infl_adj
            # have to convert to float from string because the dataframe uses row one for datatype, which has squirreled string
            dd['IRMAA-chg0'][y] = float(dd['IRMAA-chg0'][y]) * infl_adj
            dd['IRMAA-chg1'][y] = float(dd['IRMAA-chg1'][y]) * infl_adj
            dd['IRMAA-chg2'][y] = float(dd['IRMAA-chg2'][y]) * infl_adj
            dd['IRMAA-chg3'][y] = float(dd['IRMAA-chg3'][y]) * infl_adj
            dd['IRMAA-chg4'][y] = float(dd['IRMAA-chg4'][y]) * infl_adj
            dd['IRMAA-chg5'][y] = float(dd['IRMAA-chg5'][y]) * infl_adj
            dd['tax0'][y] *= infl_adj
            dd['tax1'][y] *= infl_adj
            dd['tax2'][y] *= infl_adj
            dd['tax3'][y] *= infl_adj
            dd['tax4'][y] *= infl_adj
            dd['tax5'][y] *= infl_adj
            dd['tax6'][y] *= infl_adj
            dd['cgt0'][y] *= infl_adj
            dd['cgt15'][y] *= infl_adj
            dd['SSA_income'][y] *= infl_adj
            dd['surplus'][y] *= infl_adj
        # now lop off year 0
        for k,v in dd.items():
            dd[k] = v[1:]
        # adjust relative offsets
        set_nut(dd, 'eoplan_spouse', get_nut(dd, 'eoplan_spouse') - 1)

    # 'initial_DI': 0, 'smallest_DI': 0, 'mean_DI': 0, 'sdtdev_DI': 0, 'FTAB': 0, 'sum_DI': 0
    sm['initial_DI'] = rs['disp_income'][1]
    sm['smallest_DI'] = min(rs['disp_income'][1:i+1])
    sm['mean_DI'] = statistics.mean(rs['disp_income'][1:i+1])
    sm['sdtdev_DI'] = statistics.stdev(rs['disp_income'][1:i+1])
    sm['FTAB'] = rs['net_pretax'][i+1]
    sm['sum_DI'] = sum(rs['disp_income'][1:i+1])
    
    with out_box:
        #dfr = pd.DataFrame(dd, index = dd["year"])
        #df = pd.DataFrame(dfr[1:])
        rsf = pd.DataFrame(rs, index = rs['year'])
        rf = pd.DataFrame(rsf[1:i+1])
        display(Markdown('\n### 3-Peat Real Values (Base Year $000s)'))
        display(pd.DataFrame(rf, columns=['year', 'e', 'j', 'hist_year', 'ror_stock', 'dvd_stock', 'ror_bonds', 'infl_rate',
                                          'net_pretax', 'disp_income', 'IRMAA', 'income_tax', 'withdrawals', 'x_RothConv']))
        display(sm)
        #display_dd(dd, fname)
        #
        if verbose:
            rf.to_csv(fnvers(99))
    #
    # TODO?:
    # i-ORP: At five years before the end of 3-PEAT's plan ORP's end of plan is moved back one year
    # meaning that 3-PEAT's FTAB provides a buffer in case you decide to outlive your plan.
    #
    # TODO: more hair: annual withdrawal limits; see i-ORP input instructions 
    # TODO: more hair: 3-PEAT Management Fee
    
    
########## User Controls ###########

objt = widgets.ToggleButtons(
        options=[('Max Plan Surplus (FTAB)', 'net_pretax'), ('Max Spending', 0)],
        value=0,
        disabled=False,
        description='Objective:')

mnrz = widgets.BoundedFloatText(
        value=100.0,
        min=0.0,
        max=100.0,
        step=1.0,
        description='Minimum Realized Gain (% of Basis)',
        style={'description_width': 'initial'},
        disabled=False)

tout = widgets.BoundedIntText(
        value=5,
        min=1,
        max=999,
        step=1,
        description='Time Limit on Solver (s)',
        style={'description_width': 'initial'},
        disabled=False)

glim = widgets.BoundedFloatText(
        value=0.00,
        min=0,
        max=2,
        step=0.01,
        description='Gap Goal on Solver (%)',
        style={'description_width': 'initial'},
        disabled=False)

testmode = widgets.Dropdown(
    description='Test',
    options=[('Normal Mode', 0),
             ('Artifical Losses', 1),
             ('Alternate Objective', 2),
             ('3-peat', 3),
             ('Random Walk', 4),
            ],
    value=0,
    style={'description_width': 'initial'}, # 'initial'
    disabled=False,
    indent=False
)

# mode: 0: full minlp, 1: no capital losses 2: no capgains basis averaging 3: neither

nlpomode = widgets.Dropdown(
    description='Mode',
    options=[('Normal (fastest) Mode', 0),
             ('No Capital Losses', 1),
             ('No Basis Averaging', 2),
             ('No Losses nor Averaging', 3),
             ('Full (slowest) Mode', 4),
            ],
    value=0,
    style={'description_width': 'initial'}, # 'initial'
    disabled=False,
    indent=False
)


# input widget for file name for saving dataframe
efname = widgets.Text(
    value='data/_explore.csv',
    placeholder='data/_explore.csv',
    description='With output to:',
    style={'description_width': 'initial'},
    disabled=False)

run_button = widgets.Button(description='Run Projection',
                            disabled=False,
                            layout=widgets.Layout(align_self='center'))
run_button.style.button_color = 'lightgreen'

def run_oorp(b):
    try:
        run_button.disabled=True
        run_button.description='Running...'
        run_button.style.button_color = 'yellow'
        #
        if testmode.value == 4:
            walk(nlpomode.value, tout.value, glim.value, efname.value)
        if testmode.value == 3:
            three_peat(nlpomode.value, tout.value, glim.value, efname.value)
        else:
            oorp(nlpomode.value,
                'net_pretax' if testmode.value == 2 else 0, # the default is 0
                'test_losses' if testmode.value == 1 else '',
                tout.value, glim.value, efname.value)
    except Exception as e:
        error_msg = traceback.format_exc()
        display_warning(f'Unknown exception {e} in run_oorp: {error_msg}')
        
    #
    run_button.description='Run Projection'
    run_button.style.button_color = 'lightgreen'
    run_button.disabled=False

run_button.on_click(run_oorp)

winter_lbl = widgets.Label('e-ORP Explorer',style={ 'font_weight':'bold', 'font_size':'large',})

wclcol= widgets.VBox([widgets.Label('Optimizer Controls'), nlpomode, efname, run_button])

wcrcol=widgets.VBox([widgets.Label('Developer & SCIP Controls'), testmode, tout, glim,])

display(widgets.VBox([winputs,
                      err_out,
                      widgets.GridBox([winter_lbl, widgets.Label(),
                                       wclcol, wcrcol,],
                                    layout=widgets.Layout(grid_template_columns='50% 50%',
                                                            border='1px solid black',
                                                            padding='2px 2px 8px 4px')), 

                     out_box,
                     drb_out
                     ]))


